# **4 RAG and evaluation**

This notebook contains the retrieval augmented generation process and its evaluation. Its divided by the retrieval, generation and evaluation sections. The final output is the dataset called **4_fullset_RAG.csv** with the 441 companies to use on the 5_Impact_analysis notebook.

In [ ]:
! pip install langchain_community tiktoken langchainhub sentence-transformers openpyxl faiss-cpu langchain pypdf cryptography langchain-huggingface langchain_ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.2/373.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/4

**Companies database**

This database is the output from the notebook 3_ground_truth_generation. It contains the risks domains, specific risks, conflicts with mission and explanation generated by three LLM's and the majority risks voted.
It contains 200 companies.

In [ ]:
import pandas as pd
subset_llm_emb = pd.read_csv('3_llm_answers.csv')
subset_llm_emb

,Unnamed: 0,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,...,Task1_dom_LLaMA,Task1_text_LLaMA,Task1_dom_Qwen,Task1_text_Qwen,Task1_dom_Nemotron,Task1_text_Nemotron,Task1_majority_risk_categories,Task1_majority_risk_texts,Task2_majority_risk_categories,Task2_majority_risk_texts
0,162,185,Zhejiang Geely Holding Group,"$70,356.8",0.165,$812.6,-0.14,"$94,120.3",143994,1665,...,"['Privacy & Security', 'Malicious actors', 'Hu...",['Connected vehicles leak sensitive driver dat...,"['Privacy & Security', 'Malicious actors', 'Hu...",['AI-driven supply chain optimization may leak...,"['Privacy & Security', 'Malicious actors', 'Hu...","['Vehicle data (e.g., location, driving habits...","['Privacy & Security', 'Malicious actors', 'Hu...",['Connected vehicles leak sensitive driver dat...,"['Privacy & Security', 'Malicious actors', 'Hu...","['Data leaks, cyberattacks undermine trust.', ..."
1,421,476,Power Corp. of Canada,"$33,264.8",-0.111,"$1,665.3",0.103,"$568,475.4",40300,2554,...,"['Discrimination & Toxicity', 'Misinformation'...",['AI-driven insurance risk assessment unfairly...,"['Discrimination & Toxicity', 'Privacy & Secur...",['AI risk assessment in insurance may unfairly...,"['Discrimination & Toxicity', 'Misinformation'...",['AI risk assessment in insurance unfairly bia...,"['Discrimination & Toxicity', 'Misinformation'...",['AI-driven insurance risk assessment unfairly...,"['Discrimination & Toxicity', 'Misinformation'...","['Unfairly denies coverage, biased advice unde..."
2,435,493,Vibra Energia,"$32,634.1",-0.072,$954.5,2.205,"$8,961.3",3526,724,...,"['Privacy & Security', 'AI system safety, fail...",['Exposes customer energy usage patterns to un...,"['Privacy & Security', 'Malicious actors', 'So...",['Leaking sensitive customer data through AI-d...,"['Privacy & Security', 'Misinformation', 'Mali...",['Exposes customer energy usage patterns to un...,"['Privacy & Security', 'AI system safety, fail...",['Exposes customer energy usage patterns to un...,"['Privacy & Security', 'AI system safety, fail...","['Compromises customer trust', 'Leaking sensit..."
3,100,113,Brookfield,"$95,924",0.034,"$1,130",-0.45,"$490,095",240000,127,...,"['Discrimination & Toxicity', 'Misinformation'...",['Biased investment portfolio recommendations ...,"['Discrimination & Toxicity', 'Misinformation'...",['AI-driven property valuation may unfairly di...,"['Discrimination & Toxicity', 'Misinformation'...",['AI-driven property valuation analysis unfair...,"['Discrimination & Toxicity', 'Misinformation'...",['Biased investment portfolio recommendations ...,"['Discrimination & Toxicity', 'Misinformation'...","['Biased recommendations contradict mission.',..."
4,74,85,Xiamen C&D,"$110,665.6",-0.122,"$1,057.6",1.328,"$125,003.2",62740,115,...,"['Privacy & Security', 'Malicious actors', 'Mi...",['Leaks sensitive shipment data to unauthorize...,"['Privacy & Security', 'Malicious actors', 'So...",['Leakage of sensitive shipment data through A...,"['Privacy & Security', 'Malicious actors', 'Mi...",['Exposes sensitive shipment details through v...,"['Privacy & Security', 'Malicious actors', 'Mi...",['Leaks sensitive shipment data to unauthorize...,"['Privacy & Security', 'Malicious actors', 'Mi...","['Leaking sensitive data undermines trust.', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,356,397,Xiaomi,"$38,276.8",-0.081,"$2,468.5",5.712,"$45,682.8",33627,636,...,"['Discrimination & Toxicity', 'Privacy & Secur...",['AI camera enhancements misclassify skin tone...,"['Discrimination & Toxicity', 'Misinformation'...",['AI-powered camera enhancements may dispropor...,"['Discrimination & Toxicity', 'Privacy & Secur...","[""AI-powered camera enhancements unfairly enha...","['Discrimination & Toxicity', 'Privacy & Secur...",['AI camera enhancements misclassify skin tone...,"['Discrimination & Toxicity', 'Privacy & Secur...","[""Biased product recomm

**MIT domain taxonomy**

In [ ]:
mit_risks = pd.read_excel('Missions.xlsx', sheet_name="MIT_domain")
mit_domains = "\n".join(mit_risks["Domain"])
mit_descriptions = "\n".join(
    f"{row['Domain']}: {row['Description']}" for _, row in mit_risks.iterrows()
)

## **Retrieval**

**Retrieval libraries**

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
import torch

import os
from langchain.document_loaders import PyPDFLoader

**Knowledge base**

There are two sources for the knowledge base. One is the incidents database from the 0_Exploratory_analysis. The second is the academic articles and consulting reports compiled per sector.

**Incidents**

In [ ]:
#Incidents
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
incidents_df = pd.read_csv('Knowledge_database/incidents_mit.csv')
incident_docs = []
for index, row in incidents_df.iterrows():
    # Format each row into a descriptive text string
    content = f"Incident Report: {row['title']}. Description: {row['description']}. Risk Domain: {row['Risk Domain']}."
    # Add metadata to distinguish the source
    metadata = {"source": "AIID"}
    doc = Document(page_content=content, metadata=metadata)
    incident_docs.append(doc)

In [ ]:
incident_docs

[Document(metadata={'source': 'AIID'}, page_content='Incident Report: Google’s YouTube Kids App Presents Inappropriate Content. Description: YouTube’s content filtering and recommendation algorithms exposed children to disturbing and inappropriate videos.. Risk Domain: Discrimination & Toxicity.'),
 Document(metadata={'source': 'AIID'}, page_content='Incident Report: Las Vegas Self-Driving Bus Involved in Accident. Description: A self-driving public shuttle by Keolis North America and Navya was involved in a collision with a human-driven delivery truck in Las Vegas, Nevada on its first day of service.. Risk Domain: AI system safety, failures, & limitations.'),
 Document(metadata={'source': 'AIID'}, page_content='Incident Report: Uber AV Killed Pedestrian in Arizona. Description: An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.. Risk Domain: AI system safety, failures, & limitations.'),
 Document(metadata={'source': 'AIID'}, page_conte

**Documents**

In [ ]:
#Documents
import os
from langchain.document_loaders import PyPDFLoader
from concurrent.futures import ThreadPoolExecutor

# Define your folder path
pdf_dir = 'Knowledge_database'

# List all PDF files in the folder
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]

# Function to load a single PDF
def load_pdf(pdf_file):
    full_path = os.path.join(pdf_dir, pdf_file)
    loader = PyPDFLoader(full_path)
    docs = loader.load()
    # Extract sector from filename
    # Assuming the format is "Sector - Rest of filename.pdf"
    try:
        sector = pdf_file.split(" - ")[0]
    except IndexError:
        sector = "Unknown" # Handle cases where the filename format is different

    # Add metadata to each document
    for doc in docs:
        doc.metadata["source"] = "Documents"
        doc.metadata["sector"] = sector # Add the extracted sector as metadata
    return docs

# Load each PDF using ThreadPoolExecutor for parallel loading
all_docs = []
with ThreadPoolExecutor() as executor:
    # Map the loading function to the list of PDF files
    results = executor.map(load_pdf, pdf_files)
    # Extend the all_docs list with the results from each PDF
    for docs in results:
        all_docs.extend(docs)

In [ ]:
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""] # Prioritize splitting by double newline
)
chunked_docs = text_splitter.split_documents(all_docs)

In [ ]:
chunked_docs

[Document(metadata={'producer': 'Acrobat Distiller 10.1.8 (Windows)', 'creator': 'Springer', 'creationdate': '2024-04-10T18:31:20+05:30', 'author': 'Yongxiang Zhang', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'keywords': 'Business chatbot; Dialogue system; Deep learning; Reinforcement learning; Artificial intelligence', 'moddate': '2024-05-04T09:52:01+05:30', 'subject': 'Artificial Intelligence Review, https://doi.org/10.1007/s10462-024-10744-z', 'title': 'Business chatbots with deep learning technologies: state-of-the-art, taxonomies, and future research directions', 'doi': '10.1007/s10462-024-10744-z', 'robots': 'noindex', 'source': 'Documents', 'total_pages': 63, 'page': 0, 'page_label': '1', 'sector': 'Business services'}, page_content='Vol.:(0123456789)\nArtificial Intelligence Review (2024) 57:113\nhttps://doi.org/10.1007/s10462-024-10744-z\n1 3\nBusiness chatbot

**Embeddings**

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name_qwen = "Qwen/Qwen3-Embedding-8B"
model_kwargs_qwen = {'device': 'cpu'}
encode_kwargs_qwen = {'normalize_embeddings': True}

qwen_embeddings = HuggingFaceEmbeddings(
    model_name=model_name_qwen,
    model_kwargs=model_kwargs_qwen,
    encode_kwargs=encode_kwargs_qwen
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS

batch_size = 64
#Documents
print("Embedding documents in batches...")
doc_embeddings = []
for i in tqdm(range(0, len(chunked_docs), batch_size), desc="Embedding Documents"):
    batch_texts = [doc.page_content for doc in chunked_docs[i:i+batch_size]]
    # metadata to include only 'sector' and 'source'
    batch_metadatas = [{k: v for k, v in doc.metadata.items() if k in ['sector', 'source']} for doc in chunked_docs[i:i+batch_size]]
    batch_embeddings = qwen_embeddings.embed_documents(batch_texts)
    # I append tuples of (text, embedding, metadata)
    doc_embeddings.extend(zip(batch_texts, batch_embeddings, batch_metadatas))


# Incidents
incident_embeddings = []
for i in tqdm(range(0, len(incident_docs), batch_size), desc="Embedding Incidents"):
    batch_texts = [doc.page_content for doc in incident_docs[i:i+batch_size]]
    # metadata to include only 'sector' and 'source'
    batch_metadatas = [{k: v for k, v in doc.metadata.items() if k in ['sector', 'source']} for doc in incident_docs[i:i+batch_size]]
    batch_embeddings = qwen_embeddings.embed_documents(batch_texts)
    # I append tuples of (text, embedding, metadata)
    incident_embeddings.extend(zip(batch_texts, batch_embeddings, batch_metadatas))

Embedding documents in batches...


Embedding Incidents: 100%|██████████| 14/14 [13:21<00:00, 57.23s/it]


In [ ]:
#vector databases
doc_vector_store = FAISS.from_embeddings([(text, embed) for text, embed, meta in doc_embeddings], qwen_embeddings, metadatas=[meta for text, embed, meta in doc_embeddings])
incident_vector_store = FAISS.from_embeddings([(text, embed) for text, embed, meta in incident_embeddings], qwen_embeddings, metadatas=[meta for text, embed, meta in incident_embeddings])

In [ ]:
#3,5 and 7 for combined (3 incidents + 3 documents) and 6,10 and 14 for only docs and only incidents
k_to_test = [3,5,6, 7, 10, 14]
retrievers = {}
for k in k_to_test:
    retrievers[k] = {
        "docs": doc_vector_store.as_retriever(search_kwargs={"k": k}),
        "incidents": incident_vector_store.as_retriever(search_kwargs={"k": k})
    }

In [ ]:
from tqdm.notebook import tqdm

# Iterate through different k values for retrieval
for k in k_to_test:
    print(f"\nRetrieving context for k = {k}")
    doc_retriever = retrievers[k]["docs"]
    incident_retriever = retrievers[k]["incidents"]

    # Initialize new columns for retrieved documents and incidents for the current k
    subset_llm_emb[f'retrieved_docs_k{k}'] = None
    subset_llm_emb[f'retrieved_incidents_k{k}'] = None

    # Iterate through each row in the companies_survey DataFrame for retrieval
    for index, company_row in tqdm(subset_llm_emb.iterrows(), total=len(subset_llm_emb), desc=f"Retrieving Context for k={k}"):
        # Create the query for retrieval - Making it more specific and including description
        company_name = company_row['Company']
        sector_name = company_row['Sector']
        usecases = company_row['Usecases_USC']
        description_summary = company_row['Description_summary'] # Get the description summary

        query = f"What are the potential AI risks associated with using AI for {usecases} for {company_name}, described as {description_summary}, in the {sector_name} sector?"


        try:
            # Retrieve relevant documents and incidents
            retrieved_docs = doc_retriever.invoke(query)
            retrieved_incidents = incident_retriever.invoke(query)

            # Store the retrieved information in the DataFrame
            subset_llm_emb.at[index, f'retrieved_docs_k{k}'] = retrieved_docs
            subset_llm_emb.at[index, f'retrieved_incidents_k{k}'] = retrieved_incidents
        except Exception as e:
             print(f"An error occurred during retrieval for company {company_row['Company']} with k={k}: {e}")
             subset_llm_emb.at[index, f'retrieved_docs_k{k}'] = None
             subset_llm_emb.at[index, f'retrieved_incidents_k{k}'] = None

# Display the updated DataFrame with the new columns for all k (optional, for verification)
display(subset_llm_emb.head())


Retrieving context for k = 6


Retrieving Context for k=6:   0%|          | 0/200 [00:00<?, ?it/s]


Retrieving context for k = 10


Retrieving Context for k=10:   0%|          | 0/200 [00:00<?, ?it/s]


Retrieving context for k = 14


Retrieving Context for k=14:   0%|          | 0/200 [00:00<?, ?it/s]

,Unnamed: 0,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,...,risks_missinc_docs_k7,risks_missinc_inc_k7,risks_missinc_comb_k7,question,retrieved_docs_k6,retrieved_incidents_k6,retrieved_docs_k10,retrieved_incidents_k10,retrieved_docs_k14,retrieved_incidents_k14
0,20,23,TotalEnergies,"$218,945",-0.169,"$21,384",0.042,"$283,654",102579,25,...,[],"['AI system safety, failures, & limitations']","['AI system safety, failures, & limitations', ...",Generate a comprehensive AI risk analysis for ...,[page_content='cial intelligence may aid in ri...,[page_content='Incident Report: False Negative...,[page_content='cial intelligence may aid in ri...,[page_content='Incident Report: False Negative...,[page_content='cial intelligence may aid in ri...,[page_content='Incident Report: False Negative...
1,224,252,Banco Bradesco,"$56,490.5",0.095,"$2,854.2",-0.298,"$397,255.5",79583,1049,...,[],['Discrimination & Toxicity'],"['Discrimination & Toxicity', 'Privacy & Secur...",Generate a comprehensive AI risk analysis for ...,[page_content='on AI-related risks (eg use of ...,[page_content='Incident Report: UK Bank's Voic...,[page_content='on AI-related risks (eg use of ...,[page_content='Incident Report: UK Bank's Voic...,[page_content='on AI-related risks (eg use of ...,[page_content='Incident Report: UK Bank's Voic...
2,62,70,Alibaba Group Holding,"$131,337.9",0.036,"$11,165.1",0.051,"$244,426",204891,78,...,[],[],"['Socioeconomic & Environmental', 'AI system s...",Generate a comprehensive AI risk analysis for ...,[page_content='Through our engagements with re...,[page_content='Incident Report: Amazon Alleged...,[page_content='Through our engagements with re...,[page_content='Incident Report: Amazon Alleged...,[page_content='Through our engagements with re...,[page_content='Incident Report: Amazon Alleged...
3,210,237,Delta Air Lines,"$58,048",0.148,"$4,609",2.497,"$73,644",103000,347,...,[],"['Misinformation', 'AI system safety, failures...",[],Generate a comprehensive AI risk analysis for ...,[page_content='checking the inclusion of air t...,[page_content='Incident Report: Southwest Airl...,[page_content='checking the inclusion of air t...,[page_content='Incident Report: Southwest Airl...,[page_content='checking the inclusion of air t...,[page_content='Incident Report: Southwest Airl...
4,378,423,Daiwa House Industry,"$35,999.4",-0.007,"$2,067.1",-0.093,"$43,206.7",48483,738,...,[],"['Discrimination & Toxicity', 'AI system safet...","['Discrimination & Toxicity', 'AI system safet...",Generate a comprehensive AI risk analysis for ...,[page_content='SDGs that show the highest pote...,[page_content='Incident Report: Zillow Shut Do...,[page_content='SDGs that show the highest pote...,[page_content='Incident Report: Zillow Shut Do...,[page_content='SDGs that show the highest pote...,[page_content='Incident Report: Zillow Shut Do...


To remove the document id and other contents of the objects that are not relevant to reduce overloading in the generation.

In [ ]:
import re
import ast


def process_document_string(doc_string):
    """
    Parses a string representation of a list of Document objects and extracts
    metadata and page_content for each document.
    """
    if not isinstance(doc_string, str) or doc_string == '[]':
        return []

    processed_docs = []
    # Split the string at the beginning of each "Document"
    docs_str_list = re.split(r"(?=Document\(id=)", doc_string.strip('[]'))
    # Clean up the list from the split
    docs_str_list = [d.strip(', ') for d in docs_str_list if d]

    for doc_str in docs_str_list:
        try:
            # Find the starting point of the page_content
            start_page_content = doc_str.find(", page_content='")
            if start_page_content == -1:
                continue

            # Isolate the metadata and page_content parts of the string
            metadata_part = doc_str[:start_page_content]
            page_content_part = doc_str[start_page_content + len(", page_content='"):]

            # Extract the metadata dictionary
            metadata_match = re.search(r"metadata=(\{.*?\})", metadata_part)
            if not metadata_match:
                continue

            metadata_str = metadata_match.group(1)
            metadata = ast.literal_eval(metadata_str)

            # Extract and clean the page_content
            if page_content_part.endswith("')"):
                page_content = page_content_part[:-2]
            else:
                page_content = page_content_part

            # Unescape characters for proper display
            page_content = page_content.replace('\\n', '\n').replace('\\t', '\t').replace("\\'", "'")

            processed_docs.append({
                'metadata': metadata,
                'page_content': page_content
            })
        except Exception as e:
            # In case of any parsing errors, we'll just skip that document
            pass
    return processed_docs

In [ ]:
# Columns to process
columns_to_process = [
    'retrieved_docs_k3', 'retrieved_incidents_k3',
    'retrieved_docs_k5', 'retrieved_incidents_k5',
    'retrieved_docs_k7', 'retrieved_incidents_k7',
    'retrieved_docs_k6', 'retrieved_incidents_k6',
    'retrieved_docs_k10', 'retrieved_incidents_k10',
    'retrieved_docs_k14', 'retrieved_incidents_k14'
]

# Apply the processing function to each of the columns
for col in columns_to_process:
    if col in subset_llm_emb.columns:
        subset_llm_emb[col] = subset_llm_emb[col].apply(process_document_string)

In [ ]:
subset_llm_emb.retrieved_incidents_k14.loc[1]

[{'metadata': {'source': 'AIID'},
  'page_content': 'Incident Report: 2010 Market Flash Crash. Description: A modified algorithm was able to cause dramatic price volatility and disrupted trading in the US stock exchange.. Risk Domain: AI system safety, failures, & limitations.'},
 {'metadata': {'source': 'AIID'},
  'page_content': 'Incident Report: Chinese Insurer Ping An Employed Facial Recognition to Determine Customers’ Untrustworthiness, Which Critics Alleged to Likely Make Errors and Discriminate. Description: Customers’ untrustworthiness and unprofitability were reportedly determined by Ping An, a large insurance company in China, via facial-recognition measurements of micro-expressions and body-mass indices (BMI), which critics argue was likely to make mistakes, discriminate against certain ethnic groups, and undermine its own industry.. Risk Domain: Discrimination & Toxicity.'},
 {'metadata': {'source': 'AIID'},
  'page_content': 'Incident Report: Chinese Chatbots Question Comm

In [ ]:
subset_llm_emb.retrieved_docs_k6.loc[1]

[{'metadata': {'source': 'Documents', 'sector': 'Financials'},
  'page_content': 'Greater use of AI in banks’ and insurers’ core ﬁnancial decision\nmaking\nAs described in Section 1, it is likely that banks and insurers will increasingly integrate the use\nof AI into their core business decisions around the provision of credit and insurance,\nrespectively. Doing so could help enhance their product and service offering to customers,\nand it could also improve the accuracy of their financial risk management. At the same time, it\nis important to be alert to potential risks that could arise from the deployment of AI in business\nfunctions that have a direct impact on the financial position of the firm and outcomes for\ncustomers. For example, the lack of explainability and potential autonomy of advanced AI\nmodels could – if deployed without appropriate testing, governance and risk controls – lead to\na level of financial risk-taking that is not properly understood at the time.\nA range o

In [ ]:
subset_llm_emb.retrieved_context_k5.loc[1]

[{'metadata': {'source': 'Documents', 'sector': 'Financials'},
  'page_content': 'on AI-related risks (eg use of AI models\nin various types of lending).\nPotential future system-wide exercises\nto explore AI-related risks (such\nexercises could themselves use AI,\nincluding to help better understand\nexisting risks in the financial system).\nFuture operational resilience stress\ntesting focusing on AI enabled threats.\nAI Survey evidence on firms’\nuse of AI to inform key trading\ndecisions.\nMarket and supervisory\nintelligence gathering.\nMarket data reporting (eg\nvolatility, positioning trends).\nExternal research and\ncommentary.\nAI Survey evidence on third-\nparty implementation.\nMarket share metrics for\ncommon AI service providers.\nSupervisory intelligence.\nCMORG AI Taskforce on\noperational risks to the sector\nfrom the adoption of AI.\nSupervisory intelligence.\nParticipation in G7 Cyber\nExperts Group on cyber risk.\n…and to follow developments that could lead it to ide

In [ ]:
#subset_llm_emb.to_csv('4_subset_llm_embedding.csv', index=False)

## **Generation**

The following code is for generation based on the retrieved context. It will test different variations of k-chunks. I used Llama 3.3 for the RAG.

In [ ]:
#subset_llm_rag = pd.read_csv('4_subset_llm_embedding.csv')
subset_llm_rag = subset_llm_emb.copy()

**Combination prompt**

This combination prompt is the complete version of the RAG instructions. First, it filters the extracted retrieved context to drop irrelevant context, then classificate relevant documents under a risk domain from the MIT taxonomy then from the domains with relevant context, it generates specific risks grounded in company profile, then asses if the risks conflict ith the mission, if so, it gives an explanation. It also analyses if the retrieved incidents are directly related to the company or usecases, if so, it returns the title of the incident.

In [ ]:
rag_prompt_comb = [
    {"role": "system",
    "content": """You are a AI risk analyst in the {sector} sector that outputs a single valid JSON object.
Use **only** data inside `<Inputs>`. No external knowledge or speculation.

---
### **Protocol**

Your analysis must follow these four steps for each domain in `<Risk_Taxonomy>`:

**Step 1: Domain Analysis & Evidence Filtering**
For each `<Domain>`:

- **Operational Plausibility Check (Mandatory)**:
    1. The incident or document *must* describe risks that could realistically occur in the {sector} sector and `{company}`’s operations based on <Company_Profile>.
    2. If is not related to <Company_Profile> or applicable in the {sector} sector or unsure about it, discard it.

- **Filter Incidents**: {{'source': 'AIID'}}
    Keep only if:
    - Its `Risk Domain` matches the current domain name, **and**
    - It passes the Operational Plausibility Check above.

- **Filter Documents**: {{'source': 'Documents'}}
    Keep only if:
    1. There are risks that are associated to the current `<Domain>` definition, **and**
    2. The risks pass the Operational Plausibility Check above.

- **Omit Rule**: If no plausible incidents and no plausible documents remain, do not output this domain in the JSON at all.

**Step 2: Per Included Domain, produce fields**
   - **"Risks"**: Create a list of unique risk statements (≤30 words each) that:
      • Synthesize the filtered evidence and explicitly link the risks found in the filtered evidence to a concrete harmful outcome to `{company}`’s operations, not just a vague issue.
      • Avoid generic wording: name the specific operational harm and context; never copy incident or document text verbatim.

   - **"incident_related"**: `"Yes"` if at least one of the following 2 conditions is met:
          1. **Company Match**: Any filtered incident explicitly mentions `{company}` by name.
          2. **Sector and Use Case Match**: Any filtered incident is **strictly** related to the `{sector}` sector **and** explicitly mentions one of the AI use cases from `<AI_Use_Cases>`.
          - **"incident_title"**: If `"incident_related"` is `"Yes"`, write the title (Incident report) of **one** incident that met one of the two conditions and is the closest to `{company}`’s operation.
      - If neither condition is met, set it **"incident_related"**` to "No"` and **"incident_title"** to `""` .

   - **"ConflictsMission"**: `"yes"` if any risk statements clearly conflict with the mission text; else `"no"`.

   - **"Explanation"**: if `"yes"`, produce **one concise sentence** (≤35 words) that:
       1. Quotes the **exact** mission fragment.
       2. States the **specific mechanism** and **reason** by which the risks conflict with it.
       3. Describes the **resulting harm** to the mission based on `<Company_Profile>`.
     If `"no"`, write exactly `"No conflict"`.

**Step 3: Output Rules**
   - Output one **flat** JSON object whose keys are the included domain names **exactly** as in `<Available_Domains>`. Don't include ommited domains.
   - Each domain value contains the fields in this order: `"Risks"`, `"incident_related"`, `"incident_title"`, `"ConflictsMission"`, `"Explanation"`.
   - Valid JSON only. No extra text.
"""
    },
  {
    "role": "user",
    "content": """
<Inputs>
   Generate a comprehensive AI risk analysis for the company '{company}', which operates in the '{sector}' sector.
   The company's key AI applications include: {usecases}.
   The company's mission is: "{mission}". From your risk analysis, evaluate risks that conflicts with this mission."

    <Company_Profile>
    <Description>{description}</Description>
  </Company_Profile>

  <Retrieved_Context>
    - `documents`: metadata={{'source': 'Documents'}}, page_content= Text excerpts from academic and consulting reports.
    - `incidents`: metadata={{'source': 'AIID'}}, page_content="Incident Report: "Incident Report: [Title]. Description: [Text]. Risk Domain: [Name]."
    {retrieved_context}
  </Retrieved_Context>

  <Risk_Taxonomy>
    <Available_Domains>{mit_domains}</Available_Domains>
    <Domain_Definitions>{mit_descriptions}</Domain_Definitions>
  </Risk_Taxonomy>
</Inputs>
"""
  }
]

**Only documents**

This doucments prompt is the have almost the same instructions as the combination prompt but there's no incidents steps or any mention to the incidents. First, it filters the extracted retrieved context to drop irrelevant context, then classificate relevant documents under a risk domain from the MIT taxonomy then from the domains with relevant context, it generates specific risks grounded in company profile, then asses if the risks conflict ith the mission, if so, it gives an explanation.

In [ ]:
rag_prompt_docs = [
    {"role": "system",
    "content": """You are an AI risk analyst in the {sector} sector that outputs a single valid JSON object.
Use **only** data inside `<Inputs>`. No external knowledge or speculation.

---
### **Protocol**

Your analysis must follow these four steps for each domain in `<Risk_Taxonomy>`:

**Step 1: Domain Analysis & Evidence Filtering**
For each `<Domain>`:

- **Operational Plausibility Check (Mandatory)**:
    1. The document *must* describe risks that could realistically occur in the {sector} sector and `{company}`’s operations based on <Company_Profile>.
    2. If is not related to <Company_Profile> or applicable in the {sector} sector or unsure about it, discard it.

- **Filter Documents**: {{'source': 'Documents'}}
    Keep only if:
    1. There are risks that are associated to the current `<Domain>` definition, **and**
    2. The risks pass the Operational Plausibility Check above.

- **Omit Rule**: If no plausible documents remain, do not output this domain in the JSON at all.

**Step 2: Per Included Domain, produce fields**
   - **"Risks"**: Create a list of unique risk statements (≤30 words each) that:
      • Synthesize the filtered evidence and explicitly link the risks found in the the filtered evidence to a concrete harmful outcome to `{company}`’s operations, not just a vague issue.
      • Avoid generic wording: name the specific operational harm and context; never copy incident or document text verbatim.

   - **"ConflictsMission"**: `"yes"` if any risk statements clearly conflict with the mission text; else `"no"`.

   - **"Explanation"**: if `"yes"`, produce **one concise sentence** (≤35 words) that:
       1. Quotes the **exact** mission fragment.
       2. States the **specific mechanism** and **reason** by which the risks conflict with it.
       3. Describes the **resulting harm** to the mission based on `<Company_Profile>`.
     If `"no"`, write exactly `"No conflict"`.

**Step 3: Output Rules**
   - Output one **flat** JSON object whose keys are the included domain names **exactly** as in `<Available_Domains>`. Don't include ommited domains.
   - Each domain value contains the fields in this order: `"Risks"`, `"ConflictsMission"`, `"Explanation"`.
   - Valid JSON only. No extra text.
"""
    },
  {
    "role": "user",
    "content": """
<Inputs>
   Generate a comprehensive AI risk analysis for the company '{company}', which operates in the '{sector}' sector.
   The company's key AI applications include: {usecases}.
   The company's mission is: "{mission}". From your risk analysis, evaluate risks that conflicts with this mission."

    <Company_Profile>
    <Description>{description}</Description>
  </Company_Profile>

  <Retrieved_Context>
    - `documents`: metadata={{'source': 'Documents'}}, page_content= Text excerpts from academic and consulting reports.

    {retrieved_context}
  </Retrieved_Context>

  <Risk_Taxonomy>
    <Available_Domains>{mit_domains}</Available_Domains>
    <Domain_Definitions>{mit_descriptions}</Domain_Definitions>
  </Risk_Taxonomy>
</Inputs>
"""
  }
]

**Only incidents**

This incidents prompt follows the same instructions as the combination prompt but without any reference to the documents. First, it filters the extracted retrieved context to drop irrelevant context, then generates specific risks grounded in company profile, then asses if the risks conflict with the mission, if so, it gives an explanation. It also analyses if the retrieved incidents are directly related to the company or usecases, if so, it returns the title of the incident.

In [ ]:
rag_prompt_inc = [
    {"role": "system",
    "content": """You are an AI risk analyst in the {sector} sector that outputs a single valid JSON object.
Use **only** data inside `<Inputs>`. No external knowledge or speculation.

---
### **Protocol**

Your analysis must follow these four steps for each domain in `<Risk_Taxonomy>`:

**Step 1: Domain Analysis & Evidence Filtering**
For each `<Domain>`:

- **Operational Plausibility Check (Mandatory)**:
    1. The incident *must* describe risks that could realistically occur in the {sector} sector and `{company}`’s operations based on <Company_Profile>.
    2. If is not related to <Company_Profile> or applicable in the {sector} sector or unsure about it, discard it.

- **Filter Incidents**: {{'source': 'AIID'}}
    Keep only if:
    - Its `Risk Domain` matches the current domain name, **and**
    - It passes the Operational Plausibility Check above.

- **Omit Rule**: If no plausible incidents and no plausible documents remain, do not output this domain in the JSON at all.

**Step 2: Per Included Domain, produce fields**
   - **"Risks"**: Create a list of unique risk statements (≤30 words each) that:
      • Synthesize the filtered evidence and explicitly link the risks found in the filtered evidence to a concrete harmful outcome to `{company}`’s operations, not just a vague issue.
      • Avoid generic wording: name the specific operational harm and context; never copy incident or document text verbatim.

   - **"incident_related"**: `"Yes"` if at least one of the following 2 conditions is met:
          1. **Company Match**: Any filtered incident explicitly mentions `{company}` by name.
          2. **Sector and Use Case Match**: Any filtered incident is **strictly** related to the `{sector}` sector **and** explicitly mentions one of the AI use cases from `<AI_Use_Cases>`.
          - **"incident_title"**: If `"incident_related"` is `"Yes"`, write the title (Incident report) of **one** incident that met one of the two conditions and is the closest to `{company}`’s operation.
      - If neither condition is met, set it **"incident_related"**` to "No"` and **"incident_title"** to `""` .

   - **"ConflictsMission"**: `"yes"` if any risk statements clearly conflict with the mission text; else `"no"`.

   - **"Explanation"**: if `"yes"`, produce **one concise sentence** (≤35 words) that:
       1. Quotes the **exact** mission fragment.
       2. States the **specific mechanism** and **reason** by which the risks conflict with it.
       3. Describes the **resulting harm** to the mission based on `<Company_Profile>`.
     If `"no"`, write exactly `"No conflict"`.

**Step 3: Output Rules**
   - Output one **flat** JSON object whose keys are the included domain names **exactly** as in `<Available_Domains>`. Don't include ommited domains.
   - Each domain value contains the fields in this order: `"Risks"`, `"incident_related"`, `"incident_title"`, `"ConflictsMission"`, `"Explanation"`.
   - Valid JSON only. No extra text.
"""
    },
  {
    "role": "user",
    "content": """
<Inputs>
   Generate a comprehensive AI risk analysis for the company '{company}', which operates in the '{sector}' sector.
   The company's key AI applications include: {usecases}.
   The company's mission is: "{mission}". From your risk analysis, evaluate risks that conflicts with this mission."

    <Company_Profile>
    <Description>{description}</Description>
  </Company_Profile>

  <Retrieved_Context>
    - `incidents`: metadata={{'source': 'AIID'}}, page_content="Incident Report: "Incident Report: [Title]. Description: [Text]. Risk Domain: [Name]."
    {retrieved_context}
  </Retrieved_Context>

  <Risk_Taxonomy>
    <Available_Domains>{mit_domains}</Available_Domains>
    <Domain_Definitions>{mit_descriptions}</Domain_Definitions>
  </Risk_Taxonomy>
</Inputs>
"""
  }
]

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Prompts and parsers
prompt_rag_comb = ChatPromptTemplate.from_messages(rag_prompt_comb)
prompt_rag_docs = ChatPromptTemplate.from_messages(rag_prompt_docs)
prompt_rag_inc = ChatPromptTemplate.from_messages(rag_prompt_inc)
risk_parser = JsonOutputParser()

llm_llama = ChatOllama(model="llama3.3:70b", format="json")


# Chains for each source configuration
chain_rag_comb = prompt_rag_comb | llm_llama | risk_parser
chain_rag_docs = prompt_rag_docs | llm_llama | risk_parser
chain_rag_inc = prompt_rag_inc | llm_llama | risk_parser

In [ ]:
#testing with the first row
import json
from tqdm import tqdm

# Dictionary to store results for the first company and different k values
results_first_company_k = {}

# Define the k values to use
k_to_test = [3, 5, 7]
#filter dataframe with a row
subset_llm_filtered = subset_llm_rag[subset_llm_rag['ID'] == 156]

for index, company_row in tqdm(subset_llm_filtered.iterrows(), total=len(subset_llm_filtered), desc="Processing Company with ID 110"):
    company_name = company_row["Company"].strip()
    results_first_company_k[company_name] = {}

    # Iterate through each k value
    for k in k_to_test:
        print(f"\n  Processing for k = {k}")

        description = company_row["Description_summary"].strip()
        sector = company_row["Sector"].strip()
        usecases = company_row["Usecases_USC"].strip()
        mission = company_row["Missions_validated"].strip()

        # Get the combined retrieved context from the new column
        combined_context_col = f'retrieved_context_k{k}'
        retrieved_context_str = company_row.get(combined_context_col, "")

        # The RAG prompt expects a list of strings for 'retrieved_context'
        # Since the combined column is a single string, I wrapped it in a list
        retrieved_context_list = [retrieved_context_str] if retrieved_context_str else []

        print(f"Retrieved context for k={k}: {retrieved_context_list}")

        # Invoke the single chain with the combined input
        input_rag = {
            "company":         company_name,
            "description":     description,
            "sector":        sector,
            "usecases":        usecases,
            "mission":         mission,
            "retrieved_context": retrieved_context_list, # Use the prepared list
            "mit_domains":     mit_domains,
            "mit_descriptions": mit_descriptions
        }

        try:
            # Store the result from the single chain directly under the k value
            results_first_company_k[company_name][f'k{k}'] = chain_rag_comb.invoke(input_rag)
        except Exception as e:
            results_first_company_k[company_name][f'k{k}'] = {"error": str(e)}

print(json.dumps(results_first_company_k, indent=2))

Processing Company with ID 110:   0%|          | 0/1 [00:00<?, ?it/s]


  Processing for k = 3
Retrieved context for k=3: [[{'metadata': {'source': 'Documents', 'sector': 'Transportation'}, 'page_content': 'Understanding AI Risks in Transportation | September 2024 5\nUse of AI in Transportation\nAcross\tthe\ttransportation\tsector,\tthere\tare\tmany\tpotential\tapplications\twhere\tAI\tcould\tprovide\t\nuseful\tfunctionality,\tincluding\tautomated\tvehicles,\ttraffic\tmanagement,\tdigital\tinfrastructure,\t\nand\tvehicle\tand\tinfrastructure\tmaintenance.\tThe\tspecific\trole\tthat\tAI\tplays\tin\tthe\tcontext\tof\tan\t\napplication is a major determinant of the types of risks it may pose to the transportation system. To \nhelp\tinform\tthe\tunderstanding\tof\tAI\trisk,\tthis\tsection\tprovides\tbasic\tinformation\tabout\tAI\tuses\tin\t\ntransportation,\tincluding\tcontexts,\tuse\tcases,\tand\toperational\tconcepts.\nDifferent Contexts of Potential AI Usage\nAI-enabled\tsystems\tand\tapplications\tin\tthe\ttransportation\tsystem\tinclude\tboth\tvehicles\t

Processing Company with ID 110: 100%|██████████| 1/1 [01:30<00:00, 90.43s/it]

{
  "U.S. Postal Service": {
    "k3": {
      "Privacy & Security": {
        "Risks": [
          "AI-powered cameras compromise driver privacy",
          "Biometric data collection without consent"
        ],
        "incident_related": "Yes",
        "incident_title": "Amazon Forced Deployment of AI-Powered Cameras on Delivery Drivers",
        "ConflictsMission": "yes",
        "Explanation": "Mission 'To provide trusted, safe and secure communications' conflicted with risks of compromising privacy through leaking sensitive information."
      }
    },
    "k5": {
      "AI system safety, failures, & limitations": {
        "Risks": [
          "Autonomous vehicles may fail to navigate railroad crossings",
          "AI-powered robots may be struck by trains due to navigation failure"
        ],
        "incident_related": "Yes",
        "incident_title": "Starship\u2019s Autonomous Food Delivery Robot Allegedly Stranded at Railroad Crossing in Oregon, Run over by Freight Train",

**Combination iteration**

In [ ]:
# Define the k values and context types to iterate through
k_to_test = [3, 5, 7]
context_types = {
    "comb": "rag_comb_k"
}

for k in k_to_test:
    for output_col_prefix in context_types.values():
        subset_llm_rag[f'{output_col_prefix}{k}'] = pd.Series(dtype='object')

# Iterate through all the rows of the dataset
for index, company_row in tqdm(subset_llm_rag.iterrows(), total=len(subset_llm_rag), desc="Processing All Companies"):
    company_name = company_row["Company"].strip()

    # Inputs
    base_input = {
        "company": company_name,
        "description": company_row["Description_summary"].strip(),
        "sector": company_row["Sector"].strip(),
        "usecases": company_row["Usecases_USC"].strip(),
        "mission": company_row["Missions_validated"].strip(),
        "mit_domains": mit_domains,
        "mit_descriptions": mit_descriptions
    }

    # Iterate through each k value
    for k in k_to_test:
        # Prepare context data for combination
        docs_context_str = company_row.get(f'retrieved_docs_k{k}', "")
        incidents_context_str = company_row.get(f'retrieved_incidents_k{k}', "")

        contexts_to_process = {
            "comb": ([docs_context_str] if docs_context_str else []) + ([incidents_context_str] if incidents_context_str else [])
        }

        # Combined chain
        for context_name, retrieved_context_list in contexts_to_process.items():
            output_col_name = f'{context_types[context_name]}{k*2}'

            if not retrieved_context_list:
                subset_llm_rag.loc[index, output_col_name] = json.dumps({"status": "skipped_no_context"})
                continue

            input_rag = base_input.copy()
            input_rag["retrieved_context"] = retrieved_context_list

            try:
                result = chain_rag_comb.invoke(input_rag)
                subset_llm_rag.loc[index, output_col_name] = json.dumps(result)
            except Exception as e:
                print(f"\nError processing row {index} ({company_name}) for {output_col_name}: {e}")
                subset_llm_rag.loc[index, output_col_name] = json.dumps({"error": str(e)})

**only docs and only incidents iteration**

In [ ]:
import json
from tqdm import tqdm

# Define the k values and context types to iterate through
k_to_test = [6, 10, 14]
context_types = {
    "docs": "rag_docs_k",
    "inc": "rag_inc_k",
}

for k in k_to_test:
    for output_col_prefix in context_types.values():
        subset_llm_rag[f'{output_col_prefix}{k}'] = pd.Series(dtype='object')

for index, company_row in tqdm(subset_llm_rag.iterrows(), total=len(subset_llm_rag), desc="Processing All Companies"):
    company_name = company_row["Company"].strip()

    # Inputs
    base_input = {
        "company": company_name,
        "description": company_row["Description_summary"].strip(),
        "sector": company_row["Sector"].strip(),
        "usecases": company_row["Usecases_USC"].strip(),
        "mission": company_row["Missions_validated"].strip(),
        "mit_domains": mit_domains,
        "mit_descriptions": mit_descriptions
    }

    # Iterate through each k value
    for k in k_to_test:
        # Prepare context data for docs and incidents
        docs_context_str = company_row.get(f'retrieved_docs_k{k}', "")
        incidents_context_str = company_row.get(f'retrieved_incidents_k{k}', "")

        contexts_to_process = {
            "docs": ([docs_context_str] if docs_context_str else []),
            "inc": ([incidents_context_str] if incidents_context_str else []),
        }

        # Run the chain for each context type
        for context_name, retrieved_context_list in contexts_to_process.items():
            output_col_name = f'{context_types[context_name]}{k}'

            if not retrieved_context_list:
                subset_llm_rag.loc[index, output_col_name] = json.dumps({"status": "skipped_no_context"})
                continue

            input_rag = base_input.copy()
            input_rag["retrieved_context"] = retrieved_context_list

            try:
                # Select the appropriate chain based on the type of context
                if context_name == "docs":
                    result = chain_rag_docs.invoke(input_rag)
                elif context_name == "inc":
                    result = chain_rag_inc.invoke(input_rag)
                else:
                    # Handle unexpected context_name if necessary
                    result = {"error": f"Unknown context type: {context_name}"}

                subset_llm_rag.loc[index, output_col_name] = json.dumps(result)
            except Exception as e:
                print(f"\nError processing row {index} ({company_name}) for {output_col_name}: {e}")
                subset_llm_rag.loc[index, output_col_name] = json.dumps({"error": str(e)})



# Select only the new columns to display for clarity
output_columns = [f'{prefix}{k}' for k in k_to_test for prefix in context_types.values()]
result_df = subset_llm_rag[output_columns]

# Head of the results
try:
    display(result_df.head())
except NameError:
    print(result_df.head())

Processing All Companies: 100%|██████████| 100/100 [7:35:18<00:00, 273.19s/it]


--- Processing Complete ---
Displaying the new columns for the first 5 rows:


,rag_docs_k6,rag_inc_k6,rag_docs_k10,rag_inc_k10,rag_docs_k14,rag_inc_k14
0,"{""AI system safety, failures, & limitations"": ...","{""AI system safety, failures, & limitations"": ...","{""outputs"": {""domain"": ""AI system safety, fail...","{""AI system safety, failures, & limitations"": ...","{""outputs"": {""identified_risks"": [{""risk_name""...","{""AI system safety, failures, & limitations"": ..."
1,"{""AI system safety, failures, & limitations"": ...","{""Discrimination & Toxicity"": {""Risks"": [""Faci...","{""Discrimination & Toxicity"": {""Risks"": [], ""C...","{""AI system safety, failures, & limitations"": ...","{""Company_Profile"": {""Company"": ""Power Corp. o...","{""Discrimination & Toxicity"": {""Risks"": [""AI m..."
2,"{""Privacy & Security"": {""Risks"": [""AI system v...","{""AI system safety, failures, & limitations"": ...","{""Privacy & Security"": {""Risks"": [""AI system v...","{""Discrimination & Toxicity"": {""Risks"": [""AI c...","{""outputs"": {""identified_risks"": [{""risk_type""...","{""Discrimination & Toxicity"": {""Risks"": [""AI c..."
3,"{""Socioeconomic & Environmental"": {""Risks"": [""...","{""AI system safety, failures, & limitations"": ...","{""Discrimination & Toxicity"": {""Risks"": [], ""C...","{""AI system safety, failures, & limitations"": ...","{""outputs"": {""output"": {""id"": ""Taxonomy_Mappin...","{""AI system safety, failures, & limitations"": ..."
4,"{""Misinformation"": {""Risks"": [""AI spreads fals...","{""AI system safety, failures, & limitations"": ...","{""Privacy & Security"": {""Risks"": [""AI leaks se...","{""AI system safety, failures, & limitations"": ...","{""outputs"": {""Identified_Risks"": [{""Risk_Name""...","{""AI system safety, failures, & limitations"": ..."


The code below is to process the json output into different columns with a format to compare with the LLM responses. It extracts the risks domains and specific risks from the json output. It is the same mission as in the 3_ground_truth_llm

In [ ]:
def extract_risk_data(json_string):
    """
    Parses a JSON string from a cell, extracts the domains with valid risks
    and the text of those risks.
    """
    risk_domains = []
    risk_texts = []

    # Return empty lists if the input is not a valid string
    if not isinstance(json_string, str):
        return risk_domains, risk_texts

    try:
        data = json.loads(json_string)
        # Handle cases where the JSON might be an error message
        if not isinstance(data, dict) or "error" in data or "status" in data:
            return risk_domains, risk_texts

        for domain, details in data.items():
            # Check if 'Risks' key exists, is a non-empty list, and does not contain the placeholder
            if 'Risks' in details and isinstance(details['Risks'], list) and details['Risks'] and "No sector-specific risks" not in details['Risks'][0]:
                risk_domains.append(domain)
                risk_texts.extend(details['Risks']) # Use extend to add all items from the list

    except (json.JSONDecodeError, TypeError):
        # If parsing fails or data is not a dict, return empty lists
        pass

    return risk_domains, risk_texts

Applying the function on the rag columns. Dom refers to the risks domains and text to specific risks under each risk domain.

In [ ]:
import json
# Define the k values and context types to iterate through
k_to_test = [6,10,14]
context_types = ["docs", "inc", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in subset_llm_rag.columns:
            print(f"Processing column: {source_col}")

            # Define the names for the two new columns
            dom_col = f'risks_dom_{context}_k{k}'
            text_col = f'risks_text_{context}_k{k}'

            # Apply the function and create a temporary DataFrame with the two new columns
            temp_results = subset_llm_rag[source_col].apply(extract_risk_data)

            # Assign the results back to the main DataFrame
            subset_llm_rag[dom_col] = temp_results.apply(lambda x: x[0])
            subset_llm_rag[text_col] = temp_results.apply(lambda x: x[1])


# Select only the new columns to display for clarity
output_columns = [col for col in subset_llm_rag.columns if col.startswith('risks_')]

try:
    display(subset_llm_rag[output_columns].head())
except NameError:
    print(subset_llm_rag[output_columns].head())

Processing column: rag_docs_k6
Processing column: rag_inc_k6
Processing column: rag_comb_k6
Processing column: rag_docs_k10
Processing column: rag_inc_k10
Processing column: rag_comb_k10
Processing column: rag_docs_k14
Processing column: rag_inc_k14
Processing column: rag_comb_k14

--- Processing Complete ---
Displaying the newly created columns:


,risks_dom_docs_k6,risks_text_docs_k6,risks_dom_inc_k6,risks_text_inc_k6,risks_dom_comb_k6,risks_text_comb_k6,risks_dom_docs_k10,risks_text_docs_k10,risks_dom_inc_k10,risks_text_inc_k10,...,risks_exp_comb_k14,risks_missinc_docs_k6,risks_missinc_inc_k6,risks_missinc_comb_k6,risks_missinc_docs_k10,risks_missinc_inc_k10,risks_missinc_comb_k10,risks_missinc_docs_k14,risks_missinc_inc_k14,risks_missinc_comb_k14
0,"[Discrimination & Toxicity, Socioeconomic & En...",[AI-powered product recommendation engine may ...,"[Human- Computer Interaction, AI system safety...",[Customers buy harmful products due to Amazon'...,"[AI system safety, failures, & limitations, Di...",[Amazon's AI algorithm fails to recommend func...,"[Discrimination & Toxicity, Privacy & Security...",[AI-powered product recommendation engine may ...,"[Discrimination & Toxicity, Privacy & Security...",[AI recruiting tool down-ranks female applican...,...,"['Mission ""be Earth\'s most customer-centric c...",[],"['Human- Computer Interaction', 'AI system saf...",[],[],"['Discrimination & Toxicity', 'Privacy & Secur...","['AI system safety, failures, & limitations', ...",[],"['Discrimination & Toxicity', 'Privacy & Secur...","['Human- Computer Interaction', 'Socioeconomic..."
1,"[Socioeconomic & Environmental, AI system safe...",[Environmental harm from increased energy cons...,"[AI system safety, failures, & limitations]",[AI tools fail to predict patients' conditions...,"[AI system safety, failures, & limitations]","[AI tools fail to predict COVID patients, pote...","[Privacy & Security, Socioeconomic & Environme...",[Data leaks compromise Sinopec's operational s...,"[AI system safety, failures, & limitations, Ma...","[AI tools fail to predict COVID patients, caus...",...,[],[],"['AI system safety, failures, & limitations']",[],[],"['AI system safety, failures, & limitations', ...",[],[],"['AI system safety, failures, & limitations', ...",[]
2,"[Discrimination & Toxicity, Misinformation, Pr...",[AI algorithms may lead to biased health outco...,"[AI system safety, failures, & limitations, Di...","[AI tools fail to predict patient outcomes, ca...","[AI system safety, failures, & limitations]","[AI tools fail to predict patient outcomes, ca...","[Discrimination & Toxicity, Misinformation, Pr...",[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations]","[AI tools fail to predict patient outcomes, ca...",...,"[""'Helping people live healthier lives' is con...",[],"['AI system safety, failures, & limitations', ...","['AI system safety, failures, & limitations']",[],"['AI system safety, failures, & limitations']","['AI system safety, failures, & limitations']",[],"['AI system safety, failures, & limitations', ...","['AI system safety, failures, & limitations']"
3,"[Discrimination & Toxicity, Privacy & Security...",[AI algorithms may provide biased medication r...,"[AI system safety, failures, & limitations]",[AI tools fail to predict patients' conditions...,"[AI system safety, failures, & limitations]","[AI tools fail to predict patient outcomes, ca...","[Discrimination & Toxicity, Privacy & Security...",[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations, Pr...","[AI tools fail to predict patient risks, causi...",...,"[""'Building a world of health around every ind...",[],"['AI system safety, failures, & limitations']","['AI system safety, failures, & limitations']",[],"['AI system safety, failures, & limitations']","['AI system safety, failures, & limitations']",[],"['AI system safety, failures, & limitations']","['AI system safety, failures, & limitations']"
4,"[Privacy & Security, AI system safety, failure...","[Data leaks from AI-powered vehicle tracking, ...","[AI system safety, failures, & limitations]",[Volkswagen's vehicle autonomy systems may fai...,"[AI system safety, failures, & limitations]",[Volkswagen's AI-powered vehicles may malfunct...,"[Privacy & Security, AI system safety, fail

In [ ]:
def extract_mission_conflict_data(data):
    """
    Processes data (either a JSON string or a parsed dictionary/list) from a cell,
    extracts the domains and explanations only for items where 'ConflictsMission' is 'yes'.
    """
    mission_conflict_domains = []
    mission_conflict_explanations = []

    # Handle empty or non-string inputs
    if not data:
        return mission_conflict_domains, mission_conflict_explanations

    # If the input is a string, try to parse it as JSON
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except (json.JSONDecodeError, TypeError):
            # If parsing fails, return empty lists
            return mission_conflict_domains, mission_conflict_explanations

    # If the input is a dictionary (parsed JSON)
    if isinstance(data, dict):
        # Handle cases where the dictionary might be an error or status message
        if "error" in data or "status" in data:
            return mission_conflict_domains, mission_conflict_explanations

        for domain, details in data.items():
            # Check if 'ConflictsMission' key exists and its value is 'yes'
            if isinstance(details, dict) and details.get('ConflictsMission') == 'yes':
                mission_conflict_domains.append(domain)
                # Also grab the explanation, defaulting to an empty string if not found
                mission_conflict_explanations.append(details.get('Explanation', ''))

    # If the input is a list (already processed documents/incidents)
    elif isinstance(data, list):
         pass


    return mission_conflict_domains, mission_conflict_explanations

In [ ]:
k_to_test = [6, 10, 14]
context_types = ["docs", "inc", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in subset_llm_rag.columns:
            print(f"Processing column for mission conflicts: {source_col}")

            # Define the names for the two new columns
            miss_col = f'risks_miss_{context}_k{k}'
            exp_col = f'risks_exp_{context}_k{k}'

            # Apply the function to each row of the source column
            temp_results = subset_llm_rag[source_col].apply(extract_mission_conflict_data)

            # Assign the extracted lists back to the new columns in the main DataFrame
            subset_llm_rag[miss_col] = temp_results.apply(lambda x: x[0])
            subset_llm_rag[exp_col] = temp_results.apply(lambda x: x[1])

# --- Display Results ---
print("\n--- Mission Conflict Extraction Complete ---")
print("Displaying the newly created columns for mission conflicts:")

# Select only the new columns to display for clarity
output_columns = [col for col in subset_llm_rag.columns if col.startswith('risks_miss_') or col.startswith('risks_exp_')]

# Display the head of the results
try:
    display(subset_llm_rag[output_columns].head())
except NameError:
    print(subset_llm_rag[output_columns].head())

Processing column for mission conflicts: rag_docs_k6
Processing column for mission conflicts: rag_inc_k6
Processing column for mission conflicts: rag_comb_k6
Processing column for mission conflicts: rag_docs_k10
Processing column for mission conflicts: rag_inc_k10
Processing column for mission conflicts: rag_comb_k10
Processing column for mission conflicts: rag_docs_k14
Processing column for mission conflicts: rag_inc_k14
Processing column for mission conflicts: rag_comb_k14

--- Mission Conflict Extraction Complete ---
Displaying the newly created columns for mission conflicts:


,risks_miss_docs_k6,risks_exp_docs_k6,risks_miss_inc_k6,risks_exp_inc_k6,risks_miss_comb_k6,risks_exp_comb_k6,risks_miss_docs_k10,risks_exp_docs_k10,risks_miss_inc_k10,risks_exp_inc_k10,risks_miss_comb_k10,risks_exp_comb_k10,risks_miss_docs_k14,risks_exp_docs_k14,risks_miss_inc_k14,risks_exp_inc_k14,risks_miss_comb_k14,risks_exp_comb_k14
0,"[Discrimination & Toxicity, Socioeconomic & En...",[Amazon's mission to be 'Earth’s most customer...,"[Human- Computer Interaction, AI system safety...",['be Earth’s most customer-centric company' co...,[Discrimination & Toxicity],[“be Earth’s most customer-centric company” is...,"[Discrimination & Toxicity, Privacy & Security...","[As part of Amazon, we strive to be Earth’s mo...","[Discrimination & Toxicity, Privacy & Security...",['Be Earth’s most customer-centric company' co...,"[AI system safety, failures, & limitations, Hu...",[“be Earth’s most customer-centric company” is...,"[Discrimination & Toxicity, Privacy & Security...","[, , , ]","[Discrimination & Toxicity, Privacy & Security...",[Amazon's mission 'to be Earth’s most customer...,"[Human- Computer Interaction, Socioeconomic & ...","[Mission ""be Earth's most customer-centric com..."
1,"[Socioeconomic & Environmental, AI system safe...",['Powering a better life' conflicts with envir...,"[AI system safety, failures, & limitations]",['Powering a better life' conflicted by AI har...,[],[],[Socioeconomic & Environmental],['Powering a better life' conflicts with AI re...,"[AI system safety, failures, & limitations, Ma...",[Mission 'Powering a better life' conflicts wi...,[],[],[],[],"[AI system safety, failures, & limitations, Ma...",['Powering a better life' conflicted with AI t...,[],[]
2,"[Discrimination & Toxicity, Misinformation, Pr...",[Helping people live healthier lives conflicts...,"[AI system safety, failures, & limitations, Di...",[Helping people live healthier lives' conflict...,"[AI system safety, failures, & limitations]",[Helping people live healthier lives' is confl...,"[Discrimination & Toxicity, Misinformation, Pr...",[Helping people live healthier lives' is confl...,"[AI system safety, failures, & limitations]",[Mission 'Helping people live healthier lives'...,"[AI system safety, failures, & limitations, Pr...",[Helping people live healthier lives is confli...,[],[],"[AI system safety, failures, & limitations, Hu...",[Helping people live healthier lives' is confl...,"[AI system safety, failures, & limitations, Pr...",['Helping people live healthier lives' is conf...
3,"[Discrimination & Toxicity, Privacy & Security...","[Building a world of health, AI bias conflicts...","[AI system safety, failures, & limitations]",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",[Building a world of health around every indiv...,"[Discrimination & Toxicity, Privacy & Security...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations, Pr...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",[Building a world of health around every indiv...,[],[],"[AI system safety, failures, & limitations, Pr...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",['Building a world of health around every indi...
4,[Socioeconomic & Environmental],['We engage with communities...to enhance the ...,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]


In [ ]:
def extract_mission_incident_conflicts(data):
    """
    Parses a JSON string or processes a list/dictionary from a cell and extracts the domains where
    'ConflictsMission' is 'yes' AND 'incident_related' is 'Yes'.
    """
    conflict_domains = []

    # Handle empty or non-string/non-list inputs
    if not data or (isinstance(data, str) and not data.strip()):
        return conflict_domains

    # If the input is a string, try to parse it as JSON
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except (json.JSONDecodeError, TypeError):
            # If parsing fails, return an empty list
            return conflict_domains

    # If the input is a dictionary (parsed JSON)
    if isinstance(data, dict):
        # Handle cases where the dictionary might be an error or status message
        if "error" in data or "status" in data:
            return conflict_domains

        for domain, details in data.items():
            # Check for both conditions to be true
            if isinstance(details, dict) and details.get('ConflictsMission') == 'yes' and details.get('incident_related') == 'Yes':
                conflict_domains.append(domain)

    # If the input is a list (already processed documents/incidents) - This case is less likely
    # based on the previous steps, but included for robustness.
    elif isinstance(data, list):
         # Assuming the list contains dictionaries like the parsed JSON structure
         for item in data:
             if isinstance(item, dict):
                 # Assuming the structure is like {'domain': 'Domain Name', 'ConflictsMission': 'yes', 'incident_related': 'Yes'}
                 # This part might need adjustment depending on the exact structure of the lists
                 # in the problematic columns. Based on the error, it seems the items in the list
                 # are the domain-level dictionaries directly.
                  for domain, details in item.items():
                      if isinstance(details, dict) and details.get('ConflictsMission') == 'yes' and details.get('incident_related') == 'Yes':
                           conflict_domains.append(domain)


    return conflict_domains

In [ ]:
# Define the k values and context types to iterate through
k_to_test = [6, 10, 14]
context_types = ["docs", "inc", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in subset_llm_rag.columns:
            print(f"Processing column for combined conflicts: {source_col}")

            # Define the name for the new column
            output_col_name = f'risks_missinc_{context}_k{k}'

            # Apply the function to each row of the source column
            # The result will be a Series of lists, where each list contains the domains meeting both criteria
            subset_llm_rag[output_col_name] = subset_llm_rag[source_col].apply(extract_mission_incident_conflicts)

# --- Display Results ---
print("\n--- Combined Conflict Extraction Complete ---")
print("Displaying the newly created columns for combined mission and incident conflicts:")

# Select only the new columns to display for clarity
output_columns = [col for col in subset_llm_rag.columns if col.startswith('risks_missinc_')]

# Display the head of the results
try:
    display(subset_llm_rag[output_columns].head())
except NameError:
    print(subset_llm_rag[output_columns].head())

Processing column for combined conflicts: rag_docs_k6
Processing column for combined conflicts: rag_inc_k6
Processing column for combined conflicts: rag_comb_k6
Processing column for combined conflicts: rag_docs_k10
Processing column for combined conflicts: rag_inc_k10
Processing column for combined conflicts: rag_comb_k10
Processing column for combined conflicts: rag_docs_k14
Processing column for combined conflicts: rag_inc_k14
Processing column for combined conflicts: rag_comb_k14

--- Combined Conflict Extraction Complete ---
Displaying the newly created columns for combined mission and incident conflicts:


,risks_missinc_docs_k6,risks_missinc_inc_k6,risks_missinc_comb_k6,risks_missinc_docs_k10,risks_missinc_inc_k10,risks_missinc_comb_k10,risks_missinc_docs_k14,risks_missinc_inc_k14,risks_missinc_comb_k14
0,[],"[Human- Computer Interaction, AI system safety...",[],[],"[Discrimination & Toxicity, Privacy & Security...","[AI system safety, failures, & limitations, Hu...",[],"[Discrimination & Toxicity, Privacy & Security...","[Human- Computer Interaction, Socioeconomic & ..."
1,[],"[AI system safety, failures, & limitations]",[],[],"[AI system safety, failures, & limitations, Ma...",[],[],"[AI system safety, failures, & limitations, Ma...",[]
2,[],"[AI system safety, failures, & limitations, Di...","[AI system safety, failures, & limitations]",[],"[AI system safety, failures, & limitations]","[AI system safety, failures, & limitations]",[],"[AI system safety, failures, & limitations, Hu...","[AI system safety, failures, & limitations]"
3,[],"[AI system safety, failures, & limitations]","[AI system safety, failures, & limitations]",[],"[AI system safety, failures, & limitations]","[AI system safety, failures, & limitations]",[],"[AI system safety, failures, & limitations]","[AI system safety, failures, & limitations]"
4,[],[],[],[],[],[],[],[],[]


In [ ]:
#subset_llm_rag.to_csv('subset_llm_rag5.csv', index=False)

## **Evaluation**

I evaluated the k-chunks using precision, recall and F1-score for the risk domains and BERT score for the specific risks and explanations. For the whole answer I used RAGAS answer relevancy metric.

In [ ]:
#subset_llm_eval = pd.read_csv('subset_llm_rag5.csv')
subset_llm_eval = subset_llm_rag.copy()

**Accuracy metrics**

In [ ]:
import ast
context_types = ["docs", "inc", "comb"]
k_to_test = [6, 10, 14]
rag_columns = {}
for k in k_to_test:
    for context in context_types:
        # Task 1 Columns
        rag_columns[f'risks_dom_{context}_k{k}'] = f'risks_dom_{context}_k{k}'
        # Task 2 Columns
        rag_columns[f'risks_miss_{context}_k{k}'] = f'risks_miss_{context}_k{k}'


# Ground truth columns from the LLMs ground truth
ground_truth_columns = {
    'Task1_majority_risk_categories': 'Task1_majority_risk_categories',
    'Task2_majority_risk_categories': 'Task2_majority_risk_categories',
}

metrics = {}

# Iterate through each RAG response column
for rag_key, rag_col in rag_columns.items():
    # Determine the ground truth column based on the rag_key
    if 'risks_dom' in rag_key:
        ground_truth_col = ground_truth_columns['Task1_majority_risk_categories']
        task_name = 'Task 1'
    elif 'risks_miss' in rag_key:
        ground_truth_col = ground_truth_columns['Task2_majority_risk_categories']
        task_name = 'Task 2'
    else:
        continue

    total_tp = 0
    total_fp = 0
    total_fn = 0

    # Iterate through each row in the subset_llm
    for index, row in subset_llm_eval.iterrows():
        # Get the ground truth risks
        ground_truth_risks = row[ground_truth_col]
        if pd.isna(ground_truth_risks):
            true_set = set()
        else:
            try:
                # Safely evaluate the string representation of the list
                true_list = ast.literal_eval(ground_truth_risks)
                true_set = set(true_list)
            except (ValueError, SyntaxError):
                true_set = set()

        # Get the RAG predicted risks
        rag_risks = row[rag_col]
        # The column already contains a list, so no need for ast.literal_eval here
        if not isinstance(rag_risks, list):
            pred_set = set()
        else:
            pred_set = set(rag_risks)

        # Calculate TP, FP, FN for the current row
        tp = len(pred_set.intersection(true_set))
        fp = len(pred_set.difference(true_set))
        fn = len(true_set.difference(pred_set))

        total_tp += tp
        total_fp += fp
        total_fn += fn

    # Calculate Overall Precision and Recall
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store the metrics
    metrics[f'{rag_key}_precision'] = precision
    metrics[f'{rag_key}_recall'] = recall
    metrics[f'{rag_key}_f1'] = f1

In [ ]:
print("Precision, Recall, and F1 Scores (Micro-averaged):")

# Task 1 Results
print("\nTask 1: Relevant Domains (risks_dom_*) Metrics:")
for k in k_to_test:
    print(f"\n  --- k = {k} ---")
    for context in context_types:
        rag_key = f'risks_dom_{context}_k{k}'
        precision = metrics.get(f'{rag_key}_precision', 0)
        recall = metrics.get(f'{rag_key}_recall', 0)
        f1 = metrics.get(f'{rag_key}_f1', 0)
        print(f"    Context: '{context}'")
        print(f"      Precision: {precision:.4f}")
        print(f"      Recall:    {recall:.4f}")
        print(f"      F1 Score:  {f1:.4f}")

# Task 2 Results
print("\nTask 2: Mission-Conflicting Domains (risks_miss_*) Metrics:")
for k in k_to_test:
    print(f"\n  --- k = {k} ---")
    for context in context_types:
        rag_key = f'risks_miss_{context}_k{k}'
        precision = metrics.get(f'{rag_key}_precision', 0)
        recall = metrics.get(f'{rag_key}_recall', 0)
        f1 = metrics.get(f'{rag_key}_f1', 0)
        print(f"    Context: '{context}'")
        print(f"      Precision: {precision:.4f}")
        print(f"      Recall:    {recall:.4f}")
        print(f"      F1 Score:  {f1:.4f}")

Precision, Recall, and F1 Scores (Micro-averaged):

Task 1: Relevant Domains (risks_dom_*) Metrics:

  --- k = 6 ---
    Context: 'docs'
      Precision: 0.9415
      Recall:    0.5806
      F1 Score:  0.7183
    Context: 'inc'
      Precision: 0.9443
      Recall:    0.2894
      F1 Score:  0.4430
    Context: 'comb'
      Precision: 0.9634
      Recall:    0.2998
      F1 Score:  0.4573

  --- k = 10 ---
    Context: 'docs'
      Precision: 0.9392
      Recall:    0.6157
      F1 Score:  0.7438
    Context: 'inc'
      Precision: 0.9179
      Recall:    0.3501
      F1 Score:  0.5069
    Context: 'comb'
      Precision: 0.9422
      Recall:    0.2941
      F1 Score:  0.4483

  --- k = 14 ---
    Context: 'docs'
      Precision: 0.9162
      Recall:    0.1452
      F1 Score:  0.2506
    Context: 'inc'
      Precision: 0.8731
      Recall:    0.3786
      F1 Score:  0.5281
    Context: 'comb'
      Precision: 0.9171
      Recall:    0.3463
      F1 Score:  0.5028

Task 2: Mission-Confl

**BERT score**

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
from bert_score import score
# 1. Dynamically create the dictionary with your actual column names
k_to_test = [6, 10, 14]
context_types = ["docs", "inc", "comb"]
rag_text_columns = {}
for k in k_to_test:
    for context in context_types:
        # Task 1 Text Columns
        rag_text_columns[f'risks_text_{context}_k{k}'] = f'risks_text_{context}_k{k}'
        # Task 2 Explanation Columns
        rag_text_columns[f'risks_exp_{context}_k{k}'] = f'risks_exp_{context}_k{k}'

# Define the ground truth text columns
ground_truth_text_columns = {
    'Task1_majority_risk_texts': 'Task1_majority_risk_texts',
    'Task2_majority_risk_texts': 'Task2_majority_risk_texts',
}

# Dictionary to store all results
evaluation_results = {}

# Iterate through each RAG text response column
for rag_key, rag_col in rag_text_columns.items():
    # Determine the ground truth column
    if 'risks_text' in rag_key:
        ground_truth_col = ground_truth_text_columns['Task1_majority_risk_texts']
    elif 'risks_exp' in rag_key:
        ground_truth_col = ground_truth_text_columns['Task2_majority_risk_texts']
    else:
        continue

    # Prepare lists for BERTScore and robustness calculation
    candidate_responses = []
    reference_responses = []
    valid_responses_count = 0
    total_rows = len(subset_llm_eval)

    for index, row in subset_llm_eval.iterrows():
        rag_response = row.get(rag_col)
        ground_truth_response = row.get(ground_truth_col)
        try:
            gt_list = ast.literal_eval(ground_truth_response) if pd.notna(ground_truth_response) else []
            ground_truth_response_str = ", ".join(gt_list) if isinstance(gt_list, list) else str(gt_list)
        except (ValueError, SyntaxError):
            ground_truth_response_str = str(ground_truth_response)

        rag_response_str = ", ".join(rag_response) if isinstance(rag_response, list) else str(rag_response)

        # A response is valid if the RAG output is a non-empty string.
        if rag_response_str.strip() and rag_response_str != 'nan':
            valid_responses_count += 1
            # For a valid response, check if there is a corresponding ground truth to compare against.
            if ground_truth_response_str.strip() and ground_truth_response_str != 'nan':
                candidate_responses.append(rag_response_str)
                reference_responses.append(ground_truth_response_str)

    # Calculate response rate based on valid responses
    response_rate = (valid_responses_count / total_rows) if total_rows > 0 else 0

    # Store results for the current key
    evaluation_results[rag_key] = {
        'response_rate': response_rate,
        'valid_pairs_for_bertscore': len(candidate_responses),
        'valid_responses_count': valid_responses_count, # Storing the count for correct printing
        'total_rows': total_rows,
        'precision': 0, 'recall': 0, 'f1': 0
    }

    # Calculate BERTScore only if there are valid pairs
    if candidate_responses and reference_responses:
        print(f"\nCalculating BERTScore for {rag_key} ({len(candidate_responses)} pairs)...")
        try:
            P, R, F1 = score(candidate_responses, reference_responses, lang="en", verbose=False, model_type='bert-base-uncased')
            evaluation_results[rag_key]['precision'] = P.mean().item()
            evaluation_results[rag_key]['recall'] = R.mean().item()
            evaluation_results[rag_key]['f1'] = F1.mean().item()
        except Exception as e:
            print(f"  Error calculating BERTScore for {rag_key}: {e}")


Calculating BERTScore for risks_text_docs_k6 (200 pairs)...

Calculating BERTScore for risks_exp_docs_k6 (192 pairs)...

Calculating BERTScore for risks_text_inc_k6 (200 pairs)...

Calculating BERTScore for risks_exp_inc_k6 (125 pairs)...

Calculating BERTScore for risks_text_comb_k6 (200 pairs)...

Calculating BERTScore for risks_exp_comb_k6 (104 pairs)...

Calculating BERTScore for risks_text_docs_k10 (188 pairs)...

Calculating BERTScore for risks_exp_docs_k10 (171 pairs)...

Calculating BERTScore for risks_text_inc_k10 (200 pairs)...

Calculating BERTScore for risks_exp_inc_k10 (139 pairs)...

Calculating BERTScore for risks_text_comb_k10 (200 pairs)...

Calculating BERTScore for risks_exp_comb_k10 (128 pairs)...

Calculating BERTScore for risks_text_docs_k14 (36 pairs)...

Calculating BERTScore for risks_exp_docs_k14 (28 pairs)...

Calculating BERTScore for risks_text_inc_k14 (200 pairs)...

Calculating BERTScore for risks_exp_inc_k14 (142 pairs)...

Calculating BERTScore for ris

In [ ]:
print("\n--- Comprehensive Text Evaluation Results ---")
for rag_key, metrics in evaluation_results.items():
    print(f"\n  Analysis for: {rag_key}")
    print(f"    Response Rate: {metrics['response_rate']:.2%} ({metrics['valid_responses_count']}/{metrics['total_rows']} valid responses)")
    if metrics['valid_pairs_for_bertscore'] > 0:
        print(f"    BERTScore (on {metrics['valid_pairs_for_bertscore']} pairs):")
        print(f"      Precision: {metrics['precision']:.4f}")
        print(f"      Recall:    {metrics['recall']:.4f}")
        print(f"      F1 Score:  {metrics['f1']:.4f}")
    else:
        print("    BERTScore: No valid pairs to score.")


--- Comprehensive Text Evaluation Results ---

  Analysis for: risks_text_docs_k6
    Response Rate: 100.00% (200/200 valid responses)
    BERTScore (on 200 pairs):
      Precision: 0.6704
      Recall:    0.4984
      F1 Score:  0.5710

  Analysis for: risks_exp_docs_k6
    Response Rate: 96.00% (192/200 valid responses)
    BERTScore (on 192 pairs):
      Precision: 0.6519
      Recall:    0.5146
      F1 Score:  0.5737

  Analysis for: risks_text_inc_k6
    Response Rate: 100.00% (200/200 valid responses)
    BERTScore (on 200 pairs):
      Precision: 0.6068
      Recall:    0.4267
      F1 Score:  0.5001

  Analysis for: risks_exp_inc_k6
    Response Rate: 62.50% (125/200 valid responses)
    BERTScore (on 125 pairs):
      Precision: 0.6266
      Recall:    0.4626
      F1 Score:  0.5312

  Analysis for: risks_text_comb_k6
    Response Rate: 100.00% (200/200 valid responses)
    BERTScore (on 200 pairs):
      Precision: 0.6254
      Recall:    0.4378
      F1 Score:  0.5139

  A

## **RAGAS**

i used RAGAS to evaluate the whole RAG output. RAGAS contains RAG metrics using LLM-as-a-judge approach. I selected Answer relevancy that works under the assumption that from a good answer the real question could be inferred. It works by generating three or more questions based on the answer and then compares the three question with the original question by cosine similarity average. For each k chunk and retrieval source, the RAGAs was applied.

First, I recreated the original question using the user query on the prompts. This query was ""Generate a comprehensive AI risk analysis for the company '{row['Company']}', which operates in the '{row['Sector']}' sector." For each company the question was generated by filling the fields with the company's information.

In [ ]:
def create_question(row):
    """
    The question used in the user input for the rag that combines company description,
    sector, AI use cases, and mission statement.
    """
    # Start with the base question using company name and sector
    question = f"Generate a comprehensive AI risk analysis for the company '{row['Company']}', which operates in the '{row['Sector']}' sector."

    # Safely add the company's AI use cases if they exist
    use_cases = row.get('Usecases_USC')
    if pd.notna(use_cases) and use_cases.strip():
        question += f" The company's key AI applications include: {use_cases}."

    # Safely add the company's mission statement if it exists
    mission = row.get('Missions_validated')
    if pd.notna(mission) and mission.strip():
        question += f" The company's mission is: \"{mission}\". From your risk analysis, evaluate risks that conflicts with this mission."

    return question

# Apply the function to your DataFrame to create the new 'question' column
subset_llm_eval['question'] = subset_llm_eval.apply(create_question, axis=1)

# Display an example of a fully generated question to verify
print("--- Example of a Final Generated Question ---")
print(subset_llm_eval['question'].iloc[0])

--- Example of a Final Generated Question ---
Generate a comprehensive AI risk analysis for the company 'Amazon', which operates in the 'Retailing' sector. The company's key AI applications include: AI-powered product recommendation engine, Predictive inventory management for warehouses, Chatbot-enhanced customer order tracking. The company's mission is: "As part of Amazon, we strive to be Earth’s most customer-centric company.". From your risk analysis, evaluate risks that conflicts with this mission.


In [ ]:
from langchain_ollama import ChatOllama
llm_llama = ChatOllama(model="llama3.3:70b", format="json")

In [ ]:
import json
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from langchain_core.messages import AIMessage

**Answer relevancy**

In [ ]:
def generate_questions_from_answer(generated_answer: str, llm, num_questions: int = 3) -> list:
    """
    Generates a list of questions from a given answer using a language model.
    This is the first part of the answer relevancy calculation.
    """
    if not generated_answer or generated_answer in ["[]", ""]:
        return []

    question_generation_prompt = f"""
    Generate {num_questions} questions for the given answer.
    Your output must be a single JSON object containing a single key "questions" that holds a list of the generated question strings. Valid JSON only. No extra text.

    answer: "{generated_answer}"
    """
    try:
        response_obj = llm.invoke(question_generation_prompt)

        if isinstance(response_obj, AIMessage):
            response_str = response_obj.content
        else:
            response_str = response_obj

        response_dict = json.loads(response_str)
        generated_questions = response_dict.get("questions", [])

        if not isinstance(generated_questions, list):
            return []

        return generated_questions

    except (json.JSONDecodeError, KeyError) as e:
        print(f"  - Warning: Could not parse JSON or find 'questions' key. Error: {e}")
        return []

In [ ]:
scenarios = {
    "rag_docs": "rag_docs_k",
    "rag_inc": "rag_inc_k",
    "rag_comb": "rag_comb_k"
}
k_values = [6, 10, 14]

# Enable the progress bar for pandas' apply function
tqdm.pandas()

# Loop through each scenario and k-value to generate questions
for scenario_name, answer_col_prefix in scenarios.items():
    for k in k_values:
        answer_col = f'{answer_col_prefix}{k}'
        questions_col = f'generated_questions_{scenario_name}_k{k}'

        # Check if the answer column for this scenario exists in the DataFrame
        if answer_col in subset_llm.columns:
            print(f"\n================== Generating questions for {scenario_name} (k={k}) ==================")

            # Apply the function to each row and store the list of questions in the new column
            subset_llm[questions_col] = subset_llm.progress_apply(
                lambda row: generate_questions_from_answer(row[answer_col], llm_llama),
                axis=1)

i used the same embedding used in the retrieval, qwen_embedding

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import random

def calculate_score_from_questions(original_question: str, generated_questions_str: str, embeddings_model) -> float:
    """
    Calculates the final answer relevancy score by parsing the string of
    generated questions and then comparing them to the original question.
    """
    generated_questions = []
    # Try to safely parse the string into a list.
    try:
        # Check if the input is a non-empty string before parsing
        if isinstance(generated_questions_str, str) and generated_questions_str.strip():
            generated_questions = ast.literal_eval(generated_questions_str)
    except (ValueError, SyntaxError):
        # If parsing fails, generated_questions will remain an empty list
        pass
    # If parsing failed or the list is empty, the score is 0
    if not generated_questions or not isinstance(generated_questions, list):
        return 0.0

    original_question_embedding = embeddings_model.embed_query(original_question)
    generated_questions_embeddings = embeddings_model.embed_documents(generated_questions)

    original_question_embedding = np.array(original_question_embedding).reshape(1, -1)

    similarity_scores = cosine_similarity(
        original_question_embedding,
        np.array(generated_questions_embeddings)
    )[0]

    return float(np.mean(similarity_scores))

In [ ]:
#Using a company to test the RAGAS metric
random.seed(42) # Use the same seed as the previous cell

random_index = random.randint(0, len(subset_llm_eval) - 1)
random_row = subset_llm_eval.iloc[random_index]

print(f"--- Testing Answer Relevancy for (Index: {random_index}, Company: {random_row['Company']}) ---")

# Print the generated question for this row
print(f"Original Question:\n{random_row['question']}")


rag_column_to_evaluate = 'rag_docs_k10'

generated_questions_col_name = f'generated_questions_{rag_column_to_evaluate}'

print(f"Generated Questions:\n{random_row[generated_questions_col_name]}")

if rag_column_to_evaluate in random_row and generated_questions_col_name in random_row:
    # Retrieve the generated questions string for this specific row and column
    generated_questions_str_for_row = random_row[generated_questions_col_name]

    # Calculate the score for this row and column
    answer_relevancy_score = calculate_score_from_questions(
        random_row['question'],
        generated_questions_str_for_row,
        qwen_embeddings
    )

    print(f"\nAnswer Relevancy Score for {rag_column_to_evaluate}: {answer_relevancy_score:.4f}")

else:
    print(f"\nRequired columns ('{rag_column_to_evaluate}' or '{generated_questions_col_name}') not found in the selected random row.")

--- Testing Answer Relevancy for (Index: 163, Company: Mapfre Group) ---
Original Question:
Generate a comprehensive AI risk analysis for the company 'Mapfre Group', which operates in the 'Financials' sector. The company's key AI applications include: AI for personalized insurance quotes, Fraud detection using AI analytics, AI-driven claims processing. The company's mission is: "We are by your side every step of the way, accompanying you to move forward with peace of mind, contributing to the development of a more sustainable and supportive society.
". From your risk analysis, evaluate risks that conflicts with this mission.
Generated Questions:
['What are some potential risks associated with biased AI models in insurance underwriting?', "How can the spread of misinformation through AI-powered trading platforms conflict with a company's mission to contribute to a more sustainable and supportive society?", 'What are some ways in which overreliance on AI-driven insurance decisions can le

In [ ]:
from tqdm import tqdm

scenarios = {
    "rag_docs": "rag_docs_k",
    "rag_inc": "rag_inc_k",
    "rag_comb": "rag_comb_k"
}
k_values = [6, 10, 14]
tqdm.pandas()

for scenario_name, _ in scenarios.items():
    for k in k_values:
        questions_col = f'generated_questions_{scenario_name}_k{k}'
        score_col = f'answer_relevancy_{scenario_name}_k{k}'

        if questions_col in subset_llm_eval.columns:
            print(f"\n================== Calculating scores for {scenario_name} (k={k}) ==================")

            subset_llm_eval[score_col] = subset_llm_eval.progress_apply(
                lambda row: calculate_score_from_questions(row['question'], row[questions_col], qwen_embeddings),
                axis=1
            )
        else:
            print(f"\n--- Skipping: Column '{questions_col}' not found. ---")

print("\n================== FINAL AVERAGE SCORES ==================")
for scenario_name in scenarios.keys():
    for k in k_values:
        score_col = f'answer_relevancy_{scenario_name}_k{k}'
        if score_col in subset_llm_eval.columns:
            average_score = subset_llm_eval[score_col].mean()
            print(f"Average Answer Relevancy for {scenario_name} (k={k}): {average_score:.4f}")



================== Calculating scores for rag_docs (k=6) ==================


100%|██████████| 200/200 [05:57<00:00,  1.79s/it]



================== Calculating scores for rag_docs (k=10) ==================


100%|██████████| 200/200 [05:54<00:00,  1.77s/it]



================== Calculating scores for rag_docs (k=14) ==================


...


================== Calculating scores for rag_comb (k=10) ==================


100%|██████████| 200/200 [05:58<00:00,  1.79s/it]



================== Calculating scores for rag_comb (k=14) ==================


100%|██████████| 200/200 [05:53<00:00,  1.77s/it]


================== FINAL AVERAGE SCORES ==================
Average Answer Relevancy for rag_docs (k=6): 0.4348
Average Answer Relevancy for rag_docs (k=10): 0.4558
Average Answer Relevancy for rag_docs (k=14): 0.4517
Average Answer Relevancy for rag_inc (k=6): 0.3790
Average Answer Relevancy for rag_inc (k=10): 0.3845
Average Answer Relevancy for rag_inc (k=14): 0.3898
Average Answer Relevancy for rag_comb (k=6): 0.3883
Average Answer Relevancy for rag_comb (k=10): 0.3961
Average Answer Relevancy for rag_comb (k=14): 0.3885


In [ ]:
subset_llm_eval.to_csv('4_subset_llm_ragas.csv', index=False)

## **Full sample**

The following lines apply the best RAG configuration (only docs k=10) on the full companies dataset with mission statements (441 companies).

In [ ]:
companies = pd.read_csv('1_companies_df.csv') #the output from 1_ description_and_usecases with the 441 companies with validated missions
companies

,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Validated,Missions_validated,Sector BvD,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary
0,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,1,We aim to build a better world — helping peopl...,Retail,"Walmart Inc., incorporated on October 31, 1969...",3124,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...
1,2,Amazon,"$574,785",0.118,"$30,425",-,"$527,854",1525000,2,US,...,1,"As part of Amazon, we strive to be Earth’s mos...",Retail,"Amazon.com, Inc. provides a range of products ...",3933,"Recommendation engine improvement, Logistics o...","AI-powered product recommendation engine, Pred...",AI-driven personalized shopping recommendation...,"AI-powered product recommendation engine, Pred...",Amazon is an e-commerce giant offering product...
2,3,State Grid,"$545,947.5",0.03,"$9,204.3",0.124,"$781,126.2",1361423,3953,CN,...,1,"Power Your Beautiful Life, Empower Our Beautif...",NaN,Engaged in the operation and management of ele...,122,"AI for energy grid optimization, AI-powered ou...","AI predicts power grid energy demand, Detects ...","AI for predictive maintenance of grid assets, ...","AI predicts power grid energy demand, Detects ...",State Grid operates and manages electric power...
3,4,Saudi Aramco,"$494,890.1",-0.18,"$120,699.3",-0.241,"$660,819.2",73311,3,SA,...,1,"Aramco strives to provide reliable, affordable...",Mining & Extraction,"The company is engaged in the exploration, pro...",3687,"AI enhances oil reservoir management, AI optim...","AI predicts oil reservoir depletion rates, Det...","AI for predictive maintenance of oil rigs, Opt...","AI for predictive maintenance of oil rigs, Opt...",Saudi Aramco is a leading integrated energy an...
4,5,Sinopec Group,"$429,699.7",-0.088,"$9,393.4",-0.027,"$382,688",513434,4,CN,...,1,Powering a better life,Mining & Extraction,"China Petroleum & Chemical Corporation (the ""C...",2499,"AI for catalyst optimization, AI-powered predi...","Predictive Maintenance for Refinery Equipment,...","AI optimizes refinery operations, Predicts equ...","AI optimizes refinery operations, Predicts equ...",Sinopec Group is a Chinese energy and chemical...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,495,Subaru,"$32,540.1",0.167,"$2,664.4",0.799,"$31,835.4",37693,848,JP,...,1,We aim to be a compelling company with a stron...,Transport Manufacturing,The Companyis a Japan-based company engaged in...,716,"Predictive maintenance for vehicles, AI-powere...","AI predicts vehicle maintenance needs, Enhance...","AI for predictive maintenance of vehicles, AI-...","AI for predictive maintenance of vehicles, AI-...",Subaru manufactures and sells Subaru automobil...
437,496,Air France-KLM Group,"$32,452.3",0.169,"$1,009.7",0.319,"$38,093.7",76271,772,FR,...,1,to be at the forefront of a more responsible E...,"Transport, Freight & Storage",The Company is one of the world's leading airl...,521,"AI for route optimization, AI-powered customer...","Predictive Aircraft Maintenance Scheduling, AI...","AI for flight schedule optimization, AI-powere...","Predictive Aircraft Maintenance Scheduling, AI...",Air France-KLM Group is a leading global airli...
438,497,Enbridge,"$32,349.5",-0.21,"$4,588.3",0.988,"$136,769.6",12450,792,CA,...,1,to be the first choice for energydelivery in N...,Utilities,Enbrige Inc (formerly IPL Energy Inc) is engag...,3147,"AI-enhanced pipeline inspections, AI-powered l...",AI monitors pipeline integrity for leak detect...,"AI for predictive maintenance of pipelines, AI...",AI monitors pipeline integrity for leak detect...,Enbridge operates North America's largest crud...
439,498,

In [ ]:
subset_rag = pd.read_csv('4_subset_llm_ragas.csv') #the output from RAG

,Unnamed: 0,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,...,question,generated_questions_rag_docs_k6,generated_questions_rag_docs_k10,generated_questions_rag_docs_k14,generated_questions_rag_inc_k6,generated_questions_rag_inc_k10,generated_questions_rag_inc_k14,generated_questions_rag_comb_k6,generated_questions_rag_comb_k10,generated_questions_rag_comb_k14
0,1,2,Amazon,"$574,785",0.118,"$30,425",-,"$527,854",1525000,2,...,Generate a comprehensive AI risk analysis for ...,['How might AI-powered product recommendation ...,['What are the potential risks associated with...,['What are some potential risks associated wit...,"[""What are the potential risks associated with...","[""What are some incidents where Amazon's AI-po...","[""What are some potential risks associated wit...","[""What are some potential risks associated wit...","[""What are some risks associated with Amazon's...","[""What are some risks associated with Amazon's..."
1,4,5,Sinopec Group,"$429,699.7",-0.088,"$9,393.4",-0.027,"$382,688",513434,4,...,Generate a comprehensive AI risk analysis for ...,['What are some potential environmental risks ...,"[""What are the potential risks associated with...",['What Chinese energy and chemical company use...,['What are the risks associated with AI system...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...
2,6,8,UnitedHealth Group,"$371,622",0.146,"$22,381",0.112,"$273,720",440000,19,...,Generate a comprehensive AI risk analysis for ...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some risks associated with AI syste...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...
3,7,10,CVS Health,"$357,776",0.11,"$8,344",1.011,"$249,728",259500,11,...,Generate a comprehensive AI risk analysis for ...,['What are the potential risks associated with...,['What are some potential risks associated wit...,['What is the mission of CVS Health as stated ...,['What are some potential risks associated wit...,['What are the risks associated with AI system...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some risks associated with AI syste...,['What are some potential risks associated wit...
4,8,11,Volkswagen,"$348,408.1",0.186,"$17,944.5",0.105,"$663,063.8",684025,10,...,Generate a comprehensive AI risk analysis for ...,['What are some potential risks associated wit...,"[""What are some potential risks associated wit...",['What are the primary domains affected by AI ...,"[""What are some potential risks associated wit...",['What are some potential risks associated wit...,"[""What are some potential risks associated wit...",['What are some potential risks associated wit...,['What are the potential risks associated with...,['What are some potential risks associated wit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,432,488,Luxshare Precision Industry,"$32,758.5",0.03,"$1,547.2",0.136,"$22,822.9",232585,810,...,Generate a comprehensive AI risk analysis for ...,['What are some potential risks associated wit...,['What are the potential risks associated with...,['What are the main products designed and manu...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,['What are some potential risks associated wit...,"[""What are t

The RAG took too long, so instead of running again on the 441 companies at ocne, I did only on the companies that were not used on the evaluation and then joined with the companies on the evaluation.

In [ ]:
# Define the columns to select from subset_llm
columns_to_select = [
    'ID',
    'rag_docs_k10',
    'rag_comb_k10',
    'risks_dom_docs_k10',
    'retrieved_docs_k10',
    'retrieved_incidents_k10'
    'retrieved_docs_k5',
    'retrieved_incidents_k5'
]

subset_rag = subset_rag[columns_to_select].copy()

companies = pd.merge(companies, subset_rag, on='ID', how='outer')

print("Merged DataFrame:")
display(companies.head())
print("\nShape of Merged DataFrame:", companies.shape)

Merged DataFrame:


,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,rag_docs_k10,rag_comb_k10,risks_dom_docs_k10
0,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,"Walmart Inc., incorporated on October 31, 1969...",3124,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,NaN,NaN,NaN
1,2,Amazon,"$574,785",0.118,"$30,425",-,"$527,854",1525000,2,US,...,"Amazon.com, Inc. provides a range of products ...",3933,"Recommendation engine improvement, Logistics o...","AI-powered product recommendation engine, Pred...",AI-driven personalized shopping recommendation...,"AI-powered product recommendation engine, Pred...",Amazon is an e-commerce giant offering product...,"{""Discrimination & Toxicity"": {""Risks"": [""AI-p...","{""AI system safety, failures, & limitations"": ...","['Discrimination & Toxicity', 'Privacy & Secur..."
2,3,State Grid,"$545,947.5",0.03,"$9,204.3",0.124,"$781,126.2",1361423,3953,CN,...,Engaged in the operation and management of ele...,122,"AI for energy grid optimization, AI-powered ou...","AI predicts power grid energy demand, Detects ...","AI for predictive maintenance of grid assets, ...","AI predicts power grid energy demand, Detects ...",State Grid operates and manages electric power...,NaN,NaN,NaN
3,4,Saudi Aramco,"$494,890.1",-0.18,"$120,699.3",-0.241,"$660,819.2",73311,3,SA,...,"The company is engaged in the exploration, pro...",3687,"AI enhances oil reservoir management, AI optim...","AI predicts oil reservoir depletion rates, Det...","AI for predictive maintenance of oil rigs, Opt...","AI for predictive maintenance of oil rigs, Opt...",Saudi Aramco is a leading integrated energy an...,NaN,NaN,NaN
4,5,Sinopec Group,"$429,699.7",-0.088,"$9,393.4",-0.027,"$382,688",513434,4,CN,...,"China Petroleum & Chemical Corporation (the ""C...",2499,"AI for catalyst optimization, AI-powered predi...","Predictive Maintenance for Refinery Equipment,...","AI optimizes refinery operations, Predicts equ...","AI optimizes refinery operations, Predicts equ...",Sinopec Group is a Chinese energy and chemical...,"{""Privacy & Security"": {""Risks"": [""Data leaks ...","{""AI system safety, failures, & limitations"": ...","['Privacy & Security', 'Socioeconomic & Enviro..."



Shape of Merged DataFrame: (441, 32)


In [ ]:
# Create a boolean mask to find rows that are NaN, '[]', or ''
is_empty_mask = companies['risks_dom_docs_k10'].isna() | companies['risks_dom_docs_k10'].isin(['[]', ''])

# DataFrame for rows that are empty or NaN, which means that the RAG gave an wrong format or it wasn't part of the 200 dataset of companies
subset_unseen = companies[is_empty_mask].copy()

# DataFrame for rows that are not empty or NaN
companies_rag = companies[~is_empty_mask].copy()


# Display the DataFrame with empty values
print("### DataFrame with EMPTY 'rag_docs_k10' (subset_rag) ###")
print(f"Shape: {subset_unseen.shape}")
display(subset_unseen)

print("\n" + "="*50 + "\n")

# Display the DataFrame with non-empty values
print("### DataFrame with NON-EMPTY 'rag_docs_k10' (companies_rag) ###")
print(f"Shape: {companies_rag.shape}")
display(companies_rag)

### DataFrame with EMPTY 'rag_docs_k10' (subset_rag) ###
Shape: (253, 32)


,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,rag_docs_k10,rag_comb_k10,risks_dom_docs_k10
0,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,"Walmart Inc., incorporated on October 31, 1969...",3124,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,NaN,NaN,NaN
2,3,State Grid,"$545,947.5",0.03,"$9,204.3",0.124,"$781,126.2",1361423,3953,CN,...,Engaged in the operation and management of ele...,122,"AI for energy grid optimization, AI-powered ou...","AI predicts power grid energy demand, Detects ...","AI for predictive maintenance of grid assets, ...","AI predicts power grid energy demand, Detects ...",State Grid operates and manages electric power...,NaN,NaN,NaN
3,4,Saudi Aramco,"$494,890.1",-0.18,"$120,699.3",-0.241,"$660,819.2",73311,3,SA,...,"The company is engaged in the exploration, pro...",3687,"AI enhances oil reservoir management, AI optim...","AI predicts oil reservoir depletion rates, Det...","AI for predictive maintenance of oil rigs, Opt...","AI for predictive maintenance of oil rigs, Opt...",Saudi Aramco is a leading integrated energy an...,NaN,NaN,NaN
5,7,Apple,"$383,285",-0.028,"$96,995",-0.028,"$352,583",161000,9,US,...,"Apple Inc. designs, manufactures and markets s...",2427,1. AI-assisted device optimization; 2. Smart c...,AI-powered product recommendation for iPhone u...,"AI for personalized product recommendations, A...",AI-powered product recommendation for iPhone u...,"Apple designs, manufactures, and markets consu...",NaN,NaN,NaN
9,12,Exxon Mobil,"$344,582",-0.167,"$36,010",-0.354,"$376,317",61500,12,US,...,Exxon Mobil Corporation is an international en...,2462,"AI optimizes refinery operations, AI-driven pr...",AI predicts drilling site geological formation...,AI for predictive maintenance of refinery equi...,AI predicts drilling site geological formation...,Exxon Mobil is an international energy and pet...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,487,Heineken,"$32,823.1",0.087,"$1,269.2",-0.102,"$60,479.4",89732,762,NL,...,The Company is a Netherlands-based holding com...,1930,"AI for demand forecasting, AI-driven quality c...","AI monitors brewery fermentation processes, Pr...","AI for brewery process optimization, AI-driven...","AI for brewery process optimization, AI-driven...",Heineken Holding NV is a Netherlands-based hol...,NaN,NaN,NaN
433,490,Visa,"$32,653",0.114,"$17,273",0.155,"$90,499",28800,802,US,...,Visa Inc. is a global payment technology compa...,342,"Fraud detection improvement, AI-powered consum...",AI detects fraudulent transactions in real-tim...,"AI for real-time fraud detection, Personalized...",AI detects fraudulent transactions in real-tim...,Visa facilitates global electronic funds trans...,NaN,NaN,NaN
434,491,Apollo Global Management,"$32,644",1.808,"$5,047",-,"$313,488",6855,2380,US,...,"Apollo Global Management, Inc., formerly Tango...",947,"AI for investment portfolio optimization, AI-p...","AI-driven portfolio risk forecasting, Predicti...","AI for risk assessment, AI-driven portfolio op...","AI for risk assessment, AI-driven portfolio op...",Apollo Global Management is a global alternati...,NaN,NaN,NaN
438,497,Enbridge,"$32,349.5",-0.21,"$4,588.3",0.988,"$136,769.6",12450,792,CA,...,Enbrige Inc (formerly IPL Energy Inc) is engag...,3147,"AI-enhanced pipeline inspections, AI-powered l...",AI monitors pipeline integrity for leak detect...,"AI for predictive maintenance of pipelines, AI...",AI monitors pipeline integrity for leak detect...,Enbridge operates North America's largest crud...,NaN,NaN,NaN




### DataFrame with NON-EMPTY 'rag_docs_k10' (companies_rag) ###
Shape: (188, 32)


,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,rag_docs_k10,rag_comb_k10,risks_dom_docs_k10
1,2,Amazon,"$574,785",0.118,"$30,425",-,"$527,854",1525000,2,US,...,"Amazon.com, Inc. provides a range of products ...",3933,"Recommendation engine improvement, Logistics o...","AI-powered product recommendation engine, Pred...",AI-driven personalized shopping recommendation...,"AI-powered product recommendation engine, Pred...",Amazon is an e-commerce giant offering product...,"{""Discrimination & Toxicity"": {""Risks"": [""AI-p...","{""AI system safety, failures, & limitations"": ...","['Discrimination & Toxicity', 'Privacy & Secur..."
4,5,Sinopec Group,"$429,699.7",-0.088,"$9,393.4",-0.027,"$382,688",513434,4,CN,...,"China Petroleum & Chemical Corporation (the ""C...",2499,"AI for catalyst optimization, AI-powered predi...","Predictive Maintenance for Refinery Equipment,...","AI optimizes refinery operations, Predicts equ...","AI optimizes refinery operations, Predicts equ...",Sinopec Group is a Chinese energy and chemical...,"{""Privacy & Security"": {""Risks"": [""Data leaks ...","{""AI system safety, failures, & limitations"": ...","['Privacy & Security', 'Socioeconomic & Enviro..."
6,8,UnitedHealth Group,"$371,622",0.146,"$22,381",0.112,"$273,720",440000,19,US,...,"UnitedHealth Group Incorporated, incorporated ...",2674,"Predictive patient risk analysis, AI-driven cl...","AI-driven claim processing automation, Predict...","AI for personalized health plans, AI-driven fr...","AI-driven claim processing automation, Predict...",UnitedHealth Group offers health insurance pla...,"{""Discrimination & Toxicity"": {""Risks"": [""AI a...","{""AI system safety, failures, & limitations"": ...","['Discrimination & Toxicity', 'Misinformation'..."
7,10,CVS Health,"$357,776",0.11,"$8,344",1.011,"$249,728",259500,11,US,...,CVS Health Corporation is a health solutions c...,3718,"AI-enhanced drug interactions analysis, Predic...","AI predicts patient medication adherence, Chat...",AI for personalized medication recommendations...,AI for personalized medication recommendations...,CVS Health provides health solutions through i...,"{""Discrimination & Toxicity"": {""Risks"": [""AI a...","{""AI system safety, failures, & limitations"": ...","['Discrimination & Toxicity', 'Privacy & Secur..."
8,11,Volkswagen,"$348,408.1",0.186,"$17,944.5",0.105,"$663,063.8",684025,10,DE,...,The Volkswagen Group with its headquarters in ...,2897,"Predictive maintenance for vehicles, AI-powere...","AI predicts vehicle maintenance needs, Autonom...","AI for predictive vehicle maintenance, AI-opti...","AI for predictive vehicle maintenance, AI-opti...","Volkswagen designs, manufactures, and sells pa...","{""Privacy & Security"": {""Risks"": [""Volkswagen'...","{""AI system safety, failures, & limitations"": ...","['Privacy & Security', 'AI system safety, fail..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,488,Luxshare Precision Industry,"$32,758.5",0.03,"$1,547.2",0.136,"$22,822.9",232585,810,CN,...,The Comapny is principally engaged in the rese...,259,"Enhance connector design efficiency, Optimize ...",AI predicts connector failure rates for qualit...,"AI for predictive maintenance of machinery, AI...",AI predicts connector failure rates for qualit...,"Luxshare Precision Industry designs, manufactu...","{""AI system safety, failures, & limitations"": ...","{""AI system safety, failures, & limitations"": ...","['AI system safety, failures, & limitations', ..."
435,493,Vibra Energia,"$32,634.1",-0.072,$954.5,2.205,"$8,961.3",3526,724,BR,...,"The Company was founded on November 12, 1971 i...",1348,"Forecast demand for petrol, Optimize logistics...",AI forecasts fuel demand for inventory managem...,"AI for predictive maintenance of pipelines, Op...",AI forecasts fuel demand fo

I applied the same retrieval and generation process but this time with documents k=10 and combination k=10.

**Retrieval**

In [ ]:
k_to_test = [5, 10]
retrievers = {}
for k in k_to_test:
    retrievers[k] = {
        "docs": doc_vector_store.as_retriever(search_kwargs={"k": k}),
        "incidents": incident_vector_store.as_retriever(search_kwargs={"k": k})
    }

In [ ]:
from tqdm.notebook import tqdm

# Iterate through different k values for retrieval
for k in k_to_test:
    print(f"\nRetrieving context for k = {k}")
    doc_retriever = retrievers[k]["docs"]
    incident_retriever = retrievers[k]["incidents"]

    # Initialize new columns for retrieved documents and incidents for the current k
    subset_unseen[f'retrieved_docs_k{k}'] = None
    subset_unseen[f'retrieved_incidents_k{k}'] = None

    # Iterate through each row in the companies_survey
    for index, company_row in tqdm(subset_unseen.iterrows(), total=len(subset_unseen), desc=f"Retrieving Context for k={k}"):
        # Create the query for retrieval
        company_name = company_row['Company']
        sector_name = company_row['Sector']
        usecases = company_row['Usecases_USC']
        description_summary = company_row['Description_summary'] # Get the description summary

        query = f"What are the potential AI risks associated with using AI for {usecases} for {company_name}, described as {description_summary}, in the {sector_name} sector?"


        try:
            # Retrieve relevant documents and incidents
            retrieved_docs = doc_retriever.invoke(query)
            retrieved_incidents = incident_retriever.invoke(query)

            # Store the retrieved information in the DataFrame
            subset_unseen.at[index, f'retrieved_docs_k{k}'] = retrieved_docs
            subset_unseen.at[index, f'retrieved_incidents_k{k}'] = retrieved_incidents
        except Exception as e:
             print(f"An error occurred during retrieval for company {company_row['Company']} with k={k}: {e}")
             subset_unseen.at[index, f'retrieved_docs_k{k}'] = None
             subset_unseen.at[index, f'retrieved_incidents_k{k}'] = None

display(subset_unseen.head())


Retrieving context for k = 5


Retrieving Context for k=5:   0%|          | 0/253 [00:00<?, ?it/s]


Retrieving context for k = 10


Retrieving Context for k=10:   0%|          | 0/253 [00:00<?, ?it/s]

,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Usecases_3,Usecases_USC,Description_summary,rag_docs_k10,rag_comb_k10,risks_dom_docs_k10,retrieved_docs_k5,retrieved_incidents_k5,retrieved_docs_k10,retrieved_incidents_k10
0,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,NaN,NaN,NaN,[page_content='solve new problems in an ever-c...,[page_content='Incident Report: Walmart's Bagg...,[page_content='solve new problems in an ever-c...,[page_content='Incident Report: Walmart's Bagg...
2,3,State Grid,"$545,947.5",0.03,"$9,204.3",0.124,"$781,126.2",1361423,3953,CN,...,"AI for predictive maintenance of grid assets, ...","AI predicts power grid energy demand, Detects ...",State Grid operates and manages electric power...,NaN,NaN,NaN,[page_content='130 International Energy Agency...,[page_content='Incident Report: Australian Tel...,[page_content='130 International Energy Agency...,[page_content='Incident Report: Australian Tel...
3,4,Saudi Aramco,"$494,890.1",-0.18,"$120,699.3",-0.241,"$660,819.2",73311,3,SA,...,"AI for predictive maintenance of oil rigs, Opt...","AI for predictive maintenance of oil rigs, Opt...",Saudi Aramco is a leading integrated energy an...,NaN,NaN,NaN,[page_content='12791\nApplications of AI in oi...,[page_content='Incident Report: Meta AI's Scie...,[page_content='12791\nApplications of AI in oi...,[page_content='Incident Report: Meta AI's Scie...
5,7,Apple,"$383,285",-0.028,"$96,995",-0.028,"$352,583",161000,9,US,...,"AI for personalized product recommendations, A...",AI-powered product recommendation for iPhone u...,"Apple designs, manufactures, and markets consu...",NaN,NaN,NaN,[page_content='Through our engagements with re...,[page_content='Incident Report: Apple’s AVs Re...,[page_content='Through our engagements with re...,[page_content='Incident Report: Apple’s AVs Re...
9,12,Exxon Mobil,"$344,582",-0.167,"$36,010",-0.354,"$376,317",61500,12,US,...,AI for predictive maintenance of refinery equi...,AI predicts drilling site geological formation...,Exxon Mobil is an international energy and pet...,NaN,NaN,NaN,[page_content='12785\nApplications of AI in oi...,[page_content='Incident Report: Australian Tel...,[page_content='12785\nApplications of AI in oi...,[page_content='Incident Report: Australian Tel...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
#subset_unseen.to_csv('4_comp_unseen_embedding.csv', index=False)

In [ ]:
#subset_unseen = pd.read_csv('4_comp_unseen_embedding.csv')

In [ ]:
# Columns to process
columns_to_process = [
    'retrieved_docs_k5', 'retrieved_incidents_k5',
    'retrieved_docs_k10', 'retrieved_incidents_k10'
]

# Apply the processing function to each of the columns
for col in columns_to_process:
    if col in subset_unseen.columns:
        subset_unseen[col] = subset_unseen[col].apply(process_document_string) #function defined above in the first embedding with the subset to test

**Generation**

In [ ]:
from tqdm.notebook import tqdm
import json


k_to_test = [10]

context_types = {
    "docs": "rag_docs_k",  # pure documents
    "comb": "rag_comb_k"   # combined docs + incidents
}

subset_llm_processed = subset_unseen.copy()


for k in k_to_test:
    for prefix in context_types.values():
        subset_llm_processed[f'{prefix}{k}'] = pd.Series(dtype='object')

# Iterate through all companies
for index, company_row in tqdm(subset_llm_processed.iterrows(),
                               total=len(subset_llm_processed),
                               desc="Processing All Companies"):

    company_name = company_row["Company"].strip()

    # Common inputs for all chains
    base_input = {
        "company": company_name,
        "description": company_row["Description_summary"].strip(),
        "sector": company_row["Sector"].strip(),
        "usecases": company_row["Usecases_USC"].strip(),
        "mission": company_row["Missions_validated"].strip(),
        "mit_domains": mit_domains,
        "mit_descriptions": mit_descriptions
    }

    for k in k_to_test:
        # Full docs for 'docs' context
        docs_context_str_full = company_row.get(f'retrieved_docs_k{k}', "")

        # Half + half for 'comb' context
        k_parts = k // 2
        docs_context_str_half = company_row.get(f'retrieved_docs_k{k_parts}', "")
        incidents_context_str_half = company_row.get(f'retrieved_incidents_k{k_parts}', "")

        contexts_to_process = {
            "docs": ([docs_context_str_full] if docs_context_str_full else []),
            "comb": ([docs_context_str_half] if docs_context_str_half else []) +
                    ([incidents_context_str_half] if incidents_context_str_half else [])
        }

        for context_name, retrieved_list in contexts_to_process.items():
            output_col_name = f'{context_types[context_name]}{k}'

            if not retrieved_list:
                subset_llm_processed.loc[index, output_col_name] = json.dumps({"status": "skipped_no_context"})
                continue

            input_rag = base_input.copy()
            input_rag["retrieved_context"] = retrieved_list

            try:
                if context_name == "docs":
                    result = chain_rag_docs.invoke(input_rag)
                elif context_name == "comb":
                    result = chain_rag_comb.invoke(input_rag)
                else:
                    result = {"error": f"Unknown context type: {context_name}"}

                subset_llm_processed.loc[index, output_col_name] = json.dumps(result)

            except Exception as e:
                print(f"\nError processing row {index} ({company_name}) for {output_col_name}: {e}")
                subset_llm_processed.loc[index, output_col_name] = json.dumps({"error": str(e)})

print("\n--- Processing Complete ---")
output_columns = [f'{prefix}{k}' for k in k_to_test for prefix in context_types.values()]
try:
    display(subset_llm_processed[output_columns].head())
except NameError:
    print(subset_llm_processed[output_columns].head())

Processing All Companies:   0%|          | 0/1 [00:00<?, ?it/s]


--- Processing Complete ---


,rag_docs_k10
437,"{""AI system safety, failures, & limitations"": ..."


In [ ]:
full_rag = pd.concat([companies_rag, subset_llm_processed], ignore_index=True)

The same process as in the other notebooks to extract the risks domains and specific risks from the json output

In [ ]:
def extract_risk_data(json_string):
    """
    Parses a JSON string from a cell, extracts the domains with valid risks
    and the text of those risks.
    """
    risk_domains = []
    risk_texts = []

    # Return empty lists if the input is not a valid string
    if not isinstance(json_string, str):
        return risk_domains, risk_texts

    try:
        data = json.loads(json_string)
        # Handle cases where the JSON might be an error message
        if not isinstance(data, dict) or "error" in data or "status" in data:
            return risk_domains, risk_texts

        for domain, details in data.items():
            # Check if 'Risks' key exists, is a non-empty list, and does not contain the placeholder
            if 'Risks' in details and isinstance(details['Risks'], list) and details['Risks'] and "No sector-specific risks" not in details['Risks'][0]:
                risk_domains.append(domain)
                risk_texts.extend(details['Risks']) # Use extend to add all items from the list

    except (json.JSONDecodeError, TypeError):
        # If parsing fails or data is not a dict, return empty lists
        pass

    return risk_domains, risk_texts

In [ ]:
import json
# Define the k values and context types to iterate through
k_to_test = [10]
context_types = ["docs", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in full_rag.columns:
            print(f"Processing column: {source_col}")

            # Define the names for the two new columns
            dom_col = f'risks_dom_{context}_k{k}'
            text_col = f'risks_text_{context}_k{k}'

            # Apply the function and create a temporary DataFrame with the two new columns
            temp_results = full_rag[source_col].apply(extract_risk_data)

            # Assign the results back to the main DataFrame
            full_rag[dom_col] = temp_results.apply(lambda x: x[0])
            full_rag[text_col] = temp_results.apply(lambda x: x[1])

print("\n--- Processing Complete ---")
print("Displaying the newly created columns:")

# Select only the new columns to display for clarity
output_columns = [col for col in full_rag.columns if col.startswith('risks_')]

try:
    display(full_rag[output_columns].head())
except NameError:
    print(full_rag[output_columns].head())

Processing column: rag_docs_k10
Processing column: rag_comb_k10

--- Processing Complete ---
Displaying the newly created columns:


,risks_dom_docs_k10,risks_text_docs_k10,risks_dom_comb_k10,risks_text_comb_k10,risks_miss_docs_k10,risks_exp_docs_k10,risks_miss_comb_k10,risks_exp_comb_k10,risks_missinc_docs_k10,risks_missinc_comb_k10
0,"[Discrimination & Toxicity, Privacy & Security...",[AI-powered product recommendation engine may ...,"[AI system safety, failures, & limitations, Hu...",[Amazon's AI algorithm recommends poor-quality...,"[Discrimination & Toxicity, Privacy & Security...","[As part of Amazon, we strive to be Earth’s mo...","[AI system safety, failures, & limitations, Hu...",[“be Earth’s most customer-centric company” is...,[],"[AI system safety, failures, & limitations, Hu..."
1,"[Privacy & Security, Socioeconomic & Environme...",[Data leaks compromise Sinopec's operational s...,"[AI system safety, failures, & limitations]","[AI tools fail to predict COVID patients, AI w...",[Socioeconomic & Environmental],['Powering a better life' conflicts with AI re...,[],[],[],[]
2,"[Discrimination & Toxicity, Misinformation, Pr...",[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations, Pr...",[AI-driven claim processing may reject patient...,"[Discrimination & Toxicity, Misinformation, Pr...",[Helping people live healthier lives' is confl...,"[AI system safety, failures, & limitations, Pr...",[Helping people live healthier lives is confli...,[],"[AI system safety, failures, & limitations]"
3,"[Discrimination & Toxicity, Privacy & Security...",[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations]","[AI tools fail to predict patient outcomes, ca...","[Discrimination & Toxicity, Privacy & Security...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",[Building a world of health around every indiv...,[],"[AI system safety, failures, & limitations]"
4,"[Privacy & Security, AI system safety, failure...","[Volkswagen's vehicle data leaked, AI system s...","[AI system safety, failures, & limitations]",[Volkswagen's AI system fails to recognize obs...,[],[],[],[],[],[]


In [ ]:
def extract_mission_conflict_data(data):
    """
    Processes data (either a JSON string or a parsed dictionary/list) from a cell,
    extracts the domains and explanations only for items where 'ConflictsMission' is 'yes'.
    """
    mission_conflict_domains = []
    mission_conflict_explanations = []

    # Handle empty or non-string inputs
    if not data:
        return mission_conflict_domains, mission_conflict_explanations

    # If the input is a string, try to parse it as JSON
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except (json.JSONDecodeError, TypeError):
            # If parsing fails, return empty lists
            return mission_conflict_domains, mission_conflict_explanations

    # If the input is a dictionary (parsed JSON)
    if isinstance(data, dict):
        # Handle cases where the dictionary might be an error or status message
        if "error" in data or "status" in data:
            return mission_conflict_domains, mission_conflict_explanations

        for domain, details in data.items():
            # Check if 'ConflictsMission' key exists and its value is 'yes'
            if isinstance(details, dict) and details.get('ConflictsMission') == 'yes':
                mission_conflict_domains.append(domain)
                # Also grab the explanation, defaulting to an empty string if not found
                mission_conflict_explanations.append(details.get('Explanation', ''))

    elif isinstance(data, list):
         pass


    return mission_conflict_domains, mission_conflict_explanations

In [ ]:
k_to_test = [10]
context_types = ["docs", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in full_rag.columns:
            print(f"Processing column for mission conflicts: {source_col}")

            # Define the names for the two new columns
            miss_col = f'risks_miss_{context}_k{k}'
            exp_col = f'risks_exp_{context}_k{k}'

            # Apply the function to each row of the source column
            temp_results = full_rag[source_col].apply(extract_mission_conflict_data)

            # Assign the extracted lists back to the new columns in the main DataFrame
            full_rag[miss_col] = temp_results.apply(lambda x: x[0])
            full_rag[exp_col] = temp_results.apply(lambda x: x[1])

print("\n--- Mission Conflict Extraction Complete ---")
print("Displaying the newly created columns for mission conflicts:")

# Select only the new columns to display for clarity
output_columns = [col for col in full_rag.columns if col.startswith('risks_miss_') or col.startswith('risks_exp_')]

try:
    display(full_rag[output_columns].head())
except NameError:
    print(full_rag[output_columns].head())

Processing column for mission conflicts: rag_docs_k10
Processing column for mission conflicts: rag_comb_k10

--- Mission Conflict Extraction Complete ---
Displaying the newly created columns for mission conflicts:


,risks_miss_docs_k10,risks_exp_docs_k10,risks_miss_comb_k10,risks_exp_comb_k10
0,"[Discrimination & Toxicity, Privacy & Security...","[As part of Amazon, we strive to be Earth’s mo...","[AI system safety, failures, & limitations, Hu...",[“be Earth’s most customer-centric company” is...
1,[Socioeconomic & Environmental],['Powering a better life' conflicts with AI re...,[],[]
2,"[Discrimination & Toxicity, Misinformation, Pr...",[Helping people live healthier lives' is confl...,"[AI system safety, failures, & limitations, Pr...",[Helping people live healthier lives is confli...
3,"[Discrimination & Toxicity, Privacy & Security...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",[Building a world of health around every indiv...
4,[],[],[],[]


In [ ]:
def extract_mission_incident_conflicts(data):
    """
    Parses a JSON string or processes a list/dictionary from a cell and extracts the domains where
    'ConflictsMission' is 'yes' AND 'incident_related' is 'Yes'.
    """
    conflict_domains = []

    # Handle empty or non-string/non-list inputs
    if not data or (isinstance(data, str) and not data.strip()):
        return conflict_domains

    # If the input is a string, try to parse it as JSON
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except (json.JSONDecodeError, TypeError):
            # If parsing fails, return an empty list
            return conflict_domains

    # If the input is a dictionary (parsed JSON)
    if isinstance(data, dict):
        # Handle cases where the dictionary might be an error or status message
        if "error" in data or "status" in data:
            return conflict_domains

        for domain, details in data.items():
            # Check for both conditions to be true
            if isinstance(details, dict) and details.get('ConflictsMission') == 'yes' and details.get('incident_related') == 'Yes':
                conflict_domains.append(domain)

    # If the input is a list (already processed documents/incidents) - This case is less likely
    # based on the previous steps, but included for robustness.
    elif isinstance(data, list):
         # Assuming the list contains dictionaries like the parsed JSON structure
         for item in data:
             if isinstance(item, dict):
                 # Assuming the structure is like {'domain': 'Domain Name', 'ConflictsMission': 'yes', 'incident_related': 'Yes'}
                 # This part might need adjustment depending on the exact structure of the lists
                 # in the problematic columns. Based on the error, it seems the items in the list
                 # are the domain-level dictionaries directly.
                  for domain, details in item.items():
                      if isinstance(details, dict) and details.get('ConflictsMission') == 'yes' and details.get('incident_related') == 'Yes':
                           conflict_domains.append(domain)


    return conflict_domains

In [ ]:
# Define the k values and context types to iterate through
k_to_test = [10]
context_types = ["docs", "comb"]

# Loop through all the rag_ columns that were created previously
for k in k_to_test:
    for context in context_types:
        source_col = f'rag_{context}_k{k}'

        # Check if the source column exists in the DataFrame
        if source_col in full_rag.columns:
            print(f"Processing column for combined conflicts: {source_col}")

            # Define the name for the new column
            output_col_name = f'risks_missinc_{context}_k{k}'

            # The result will be a Series of lists, where each list contains the domains meeting both criteria
            full_rag[output_col_name] = full_rag[source_col].apply(extract_mission_incident_conflicts)

print("\n--- Combined Conflict Extraction Complete ---")
print("Displaying the newly created columns for combined mission and incident conflicts:")

# Select only the new columns to display for clarity
output_columns = [col for col in full_rag.columns if col.startswith('risks_missinc_')]

try:
    display(full_rag[output_columns].head())
except NameError:
    print(full_rag[output_columns].head())

Processing column for combined conflicts: rag_docs_k10
Processing column for combined conflicts: rag_comb_k10

--- Combined Conflict Extraction Complete ---
Displaying the newly created columns for combined mission and incident conflicts:


,risks_missinc_docs_k10,risks_missinc_comb_k10
0,[],"[AI system safety, failures, & limitations, Hu..."
1,[],[]
2,[],"[AI system safety, failures, & limitations]"
3,[],"[AI system safety, failures, & limitations]"
4,[],[]


In [ ]:
# Create a boolean mask to find rows that are NaN, '[]', or ''
# Also check if the value is a list and if it's empty
is_empty_mask = full_rag['risks_dom_docs_k10'].isna() | \
                full_rag['risks_dom_docs_k10'].isin(['', '[]']) | \
                full_rag['risks_dom_docs_k10'].apply(lambda x: isinstance(x, list) and len(x) == 0)


# DataFrame for rows that are empty or NaN

subset_unseen = full_rag[is_empty_mask].copy()

# DataFrame for rows that are not empty or NaN
companies_rag = full_rag[~is_empty_mask].copy()


print("### DataFrame with EMPTY 'rag_docs_k10' (subset_rag) ###")
print(f"Shape: {subset_unseen.shape}")
display(subset_unseen)

print("\n" + "="*50 + "\n")

print("### DataFrame with NON-EMPTY 'rag_docs_k10' (companies_rag) ###")
print(f"Shape: {companies_rag.shape}")
display(companies_rag)

### DataFrame with EMPTY 'rag_docs_k10' (subset_rag) ###
Shape: (0, 45)


,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,retrieved_incidents_k10,risks_text_docs_k10,risks_dom_comb_k10,risks_text_comb_k10,risks_miss_docs_k10,risks_exp_docs_k10,risks_miss_comb_k10,risks_exp_comb_k10,risks_missinc_docs_k10,risks_missinc_comb_k10




### DataFrame with NON-EMPTY 'rag_docs_k10' (companies_rag) ###
Shape: (441, 45)


,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,retrieved_incidents_k10,risks_text_docs_k10,risks_dom_comb_k10,risks_text_comb_k10,risks_miss_docs_k10,risks_exp_docs_k10,risks_miss_comb_k10,risks_exp_comb_k10,risks_missinc_docs_k10,risks_missinc_comb_k10
0,2,Amazon,"$574,785",0.118,"$30,425",-,"$527,854",1525000,2,US,...,NaN,[AI-powered product recommendation engine may ...,"[AI system safety, failures, & limitations, Hu...",[Amazon's AI algorithm recommends poor-quality...,"[Discrimination & Toxicity, Privacy & Security...","[As part of Amazon, we strive to be Earth’s mo...","[AI system safety, failures, & limitations, Hu...",[“be Earth’s most customer-centric company” is...,[],"[AI system safety, failures, & limitations, Hu..."
1,5,Sinopec Group,"$429,699.7",-0.088,"$9,393.4",-0.027,"$382,688",513434,4,CN,...,NaN,[Data leaks compromise Sinopec's operational s...,"[AI system safety, failures, & limitations]","[AI tools fail to predict COVID patients, AI w...",[Socioeconomic & Environmental],['Powering a better life' conflicts with AI re...,[],[],[],[]
2,8,UnitedHealth Group,"$371,622",0.146,"$22,381",0.112,"$273,720",440000,19,US,...,NaN,[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations, Pr...",[AI-driven claim processing may reject patient...,"[Discrimination & Toxicity, Misinformation, Pr...",[Helping people live healthier lives' is confl...,"[AI system safety, failures, & limitations, Pr...",[Helping people live healthier lives is confli...,[],"[AI system safety, failures, & limitations]"
3,10,CVS Health,"$357,776",0.11,"$8,344",1.011,"$249,728",259500,11,US,...,NaN,[AI algorithms may perpetuate biases in health...,"[AI system safety, failures, & limitations]","[AI tools fail to predict patient outcomes, ca...","[Discrimination & Toxicity, Privacy & Security...",[Building a world of health around every indiv...,"[AI system safety, failures, & limitations]",[Building a world of health around every indiv...,[],"[AI system safety, failures, & limitations]"
4,11,Volkswagen,"$348,408.1",0.186,"$17,944.5",0.105,"$663,063.8",684025,10,DE,...,NaN,"[Volkswagen's vehicle data leaked, AI system s...","[AI system safety, failures, & limitations]",[Volkswagen's AI system fails to recognize obs...,[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,421,Jardine Matheson,"$36,049",-0.044,$686,0.938,"$89,182",443000,668,CN,...,"[{'metadata': {'source': 'AIID'}, 'page_conten...","[AI component failure in vehicles, Lack of rob...","[AI system safety, failures, & limitations]",[Autonomous vehicles may fail to recognize obs...,[Socioeconomic & Environmental],"['Our aim is to deliver superior, long-term re...",[],[],[],[]
437,441,Paccar,"$35,127.4",0.219,"$4,600.8",0.528,"$40,823.4",32400,714,US,...,"[{'metadata': {'source': 'AIID'}, 'page_conten...",[Predictive maintenance AI fails to detect eng...,"[AI system safety, failures, & limitations]",[Autonomous vehicles may fail to recognize obs...,"[Privacy & Security, Socioeconomic & Environme...",['Be a global technology leader' conflicts wit...,[],[],[],[]
438,462,Aisin,"$33,969.6",0.044,$628.3,1.258,"$30,703.7",115140,806,JP,...,"[{'metadata': {'source': 'AIID'}, 'page_conten...",[Component failure rates affect vehicle safety...,"[AI system safety, failures, & limitations]",[Tesla's Autopilot fails to recognize parked v...,"[AI system safety, failures, & limitations]",[Mission 'deliver beauty to our future earth' ...,"[AI system safety, failures, & limitations]",[Mission 'bring freedom and happiness to movem...,[],"[AI system safety, failures, & limitations]"
439,474,Mazda Motor,"$33,403",0.182,"$1,437.1",0.362,"$25,074.5",48685,827,JP,...,"[{'metadata': {'source': 'AIID'}, 'page_conten...","[AI system failure leads to vehicle accidents,...","[AI system safety, failures, & limitations]","[Vehicle accidents due to AI syste

**Final output**

The full dataset with the 441 companies and the RAG responses to be used on the 5_Impact_analysis.

In [ ]:
full_rag.to_csv('4_fullset_RAG.csv', index=False)